# Distance modulus computation using the NIR template

#### I have to run this notebook 2 times: 
1) The first time to determine:
- the apparent magnitudes (and their uncertainties, sigma_m) at t_Bmax infered from the template fit.
- uncertainty in the photometric distance moduli of the SNe sample defined as: 
    error_mu_photometric^2 = sigma_m^2
where sigma_m is the uncertainty in the apparent magnitude at t_Bmax computed in this first run.
- the absolute magnitude for each SN from AbsMag = appMag - mu_LCDM(z).
- the average absolute magnitude at t_Bmax, mean_AbsMag, with its standard deviation of the SNe sample. Write down these values in (AverageAbsMag_atMax, err_AverageAbsMag_atMax).

Run this notebook until the end of section "Determining average Absolute magnitude of the sample".
Set: AbsMagFromHisto = False

2) The second time to determine: 
- the photometric distance moduli of the SNe sample defined as mu_photometric = appMag - mean_AbsMag.
- the intrinsic dispersion of the Hubble-diagram residual, sigma_intrinsic. 

Run the entire notebook
Set: AbsMagFromHisto = True
    
#### Diverse

Relationship between index (a.k.a, row or line) of the "Template_phase_mu_tau_FromR.dat" and the day (= phase):
day = (index - 71)/2

-----

# User interface

In [2]:
HoFix = 73.24 # Valued used by default in the paper
# HoFix = 72.0 # 72 # 73.24  # Hubble constant (km/(s Mpc))
# HoFix = 72.78 # # TEMPORAL: value reported by Dhawan et al 2017.

# Peculiar velocity (km/s). Options: (150, 200, 300, 400)
# This number must be an integer: it is used to name the output folder.
vpecFix = 150   

BandName = 'Y'   # What band to fit:(Y, J, H ,K)

# Mean Absolute magnitude determined from histogram of 'appMagTmax_s - mu_s'?:
# First run the notebook with this option with setting the value to 
# "False" in order to determine 
# the mean abs mag, then once I get that number, run a second time 
# the notebook with this option as "True"
# to compute the final tables and results.
# If "True" it is generated the final 'DistanceMu_Good_AfterCutoffs_Main_.txt' 
# text file, otherwise if "False"
# generate temporal text files.

AbsMagFromHisto = False  

# Use all this notebook just to plot the Hubble diagram?
# Useful to create the HD for other cases  different to the template method.
# If 'False' then compute the distance modulus using the template method.
NotebookToPlotOnly = True

NotebookName = '11_DistanceMu_HubbleDiagram.ipynb'    

###############################################################

#--- Fixed values ---
OmMFix = 0.28 # Omega_Matter
OmLFix = 0.72 # Omega_Lambda
wFix = -1 # Dark energy EoS
c = 299792.458  # Speed of light (km/s)
# In the process to convert all the 'c' to 'cc'
cc = 299792.458  # Speed of light (km/s) 

#--- Uncertainty in z_CMB:---

# Used to plot the -theoretical- peculiar velocity uncertanty in the
# Hubble residual plot.

# Dan Scolnic gave me the value of err_cz_CMB = 150 km/s about the 
# collection of z_CMB values he provided me.
# err_zCMB_fix =  0.0005003461427972281 when err_cz_CMB = 150 km/s
err_zCMB_fix = 150.0/cc  # 150 has units of km/s

# Just to plot the peculiar velocity uncertainty curve in the 
# Hubble-diagram residual, 
# I was using,  err_zCMB_fix = 0.001, that is the average 
# error_zcmb in Andy's compilation.

#--------------------------------------------

#   SELECT THE APPARENT MAG DATA TO USE TO CONSTRUCT THE HUBBLE DIAGRAM
# The plots with cuts z>0 and z>0.01 are done automatically.

# KindOfData4HD = 'CfA'
# KindOfData4HD = 'CSP'
# KindOfData4HD = 'Others'
KindOfData4HD = 'AllSamples'

#--------------------------------------------

#   SELECT THE TEMPLATE TO USE TO COMPUTE THE DISTANCE MODULUS

# This will be also the data used to construct the Hubble diagram
# Selecting from '3_Template' folder

#- Use the template constructed from the subsample?:
# KindOfTemp = 'CfA'
# KindOfTemp = 'CSP'
# KindOfTemp = 'Others'
KindOfTemp = 'AllSamples'

KindOfTempSubgroup = 'z_gr_0'
# KindOfTempSubgroup = 'z_gr_001'
# KindOfTempSubgroup = 'AllGoodData'

# Indicate the technique used to compute the GP fitting in 
# the '1_AllData_InitialFit' step:
# TempPrior_Hyper = Using a template prior (computed from 
# the Moving Windows Average template) for the Gaussian Process 
# fitting, then determining the hyperparameters using all 
# the LCs simultaneously.
# FlatPrior= Assuming a flat prior at ~ -17 Abs mag, then 
# computing the hyperparameters for each LC independently.
# MWA = Moving windows average

# TempType = 'TempPrior_Hyper' 
TempType = 'FlatPrior' # I use this option for the paper
# TempType = 'MWA' 

# - Smoothed Moving windows average to construct the template
# TempType = 'MWA' # Moving windows average
# If TempType = 'MWA' then specify the template file to use
MWATempTypeFile = 'TempWeightedSmooth_Box7_Step05_Window21_Poly3.dat'

# Use a build-in normalized template?:
# Normalized = True
# It is, use a template that was constructed assuming 
# vPec=0 km/s during the GP fitting of the individual LCs, 
# and then with the option 'NormalizedTemp' in the 
# hierarchical Bayesian code.

In [3]:
# Reset the values:
Average_NIRAbsMag_TBmax = None;  
error_Average_NIRAbsMag_TBmax = None;

#--------------------------------------

if BandName == 'J':
    
    # Redshift cutoff.
    # In the low-z paper we use zcmbUpperLim = 0.04.
    zcmbUpperLim = 0.04      # Options: (0.04,0.06,0.09, 0.65). 

    # These values have to be computed the first time I run this notebook.
    # NOTE: The values of "Average_NIRAbsMag_TBmax" and 
    # "error_Average_NIRAbsMag_TBmax" do NOT depend on the value of "vpecFix".

    if zcmbUpperLim == 0.04:
        
        # ------------------------------
        #   J band | vpecFix = 150 km/s | 0 < z < 0.04
        # Average_NIRAbsMag_TBmax = -18.3930007685;  # 2018-04-26; 01:29 hrs.
        # error_Average_NIRAbsMag_TBmax = 0.182603385426;
        
        # ------------------------------
        #   J band | vpecFix = 150 km/s | 0 < z < 0.04
        Average_NIRAbsMag_TBmax = -18.338193359;  # 2018-07-12; 21:34 hrs.
        error_Average_NIRAbsMag_TBmax = 0.176009661876;

    elif zcmbUpperLim == 0.06:
        # Using Foley + Cepheid + Special cases's zcmb: 
        Average_NIRAbsMag_TBmax = -18.3915728254;   
        error_Average_NIRAbsMag_TBmax = 0.177460478042; 

    elif zcmbUpperLim == 0.09:
        # Using Foley + Cepheid + Special cases's zcmb
        Average_NIRAbsMag_TBmax = nan ;   error_Average_NIRAbsMag_TBmax = nan ; #

        
    # Ignore data with a chi^2_dof larger than a given threshold:
    # Use the values ( chi2_dof_Max = 1e6; chi2_dof_Max_Label = 'chi_1e6') 
    # the first time I run the notebook.
    chi2_dof_Max = 1e6; chi2_dof_Max_Label = 'chi_1e6'  
    # chi2_dof_Max = 3.5; chi2_dof_Max_Label = 'chi3_5'

    # Length hyperparameter from the GP fit.
    l_kern = 7.0552; 

    # PHASE RANGE IN DAYS OF TEMPLATE TO USE TO COMPUTE THE DISTANCE MODULUS
    PhaseMinTemp = -8 # -8 days
    PhaseMaxTemp = 30 # 30, 41, days

    # Not in use yet. AbsMag_NIRmax= -18.4670;  error_AbsMag_NIRmax=0.02974; phase_NIRmax = -4.5; 

    #-- EBV cutoff
    EBVhostMin = -0.4 # -0.4 # host galaxy
    EBVhostMax = 0.4 # 0.4 # host galaxy. 
    EBVMWLim = 1 # Milky-Way galaxy

    #-- dm15 cutoff
    dm15LowerLim = 0.8 # I assume 0.79.
    dm15UpperLim = 1.6
    
    # - Smoothed Moving windows average to construct the template
    # If TempType = 'MWA' then specify the template file to use
    MWATempTypeFile = 'TempWeightedSmooth_Box7_Step05_Window21_Poly3.dat'

###########################################################

if BandName == 'Y': 

    # Redshift cutoff.
    # In the low-z paper we use zcmbUpperLim = 0.04.
    zcmbUpperLim = 0.04      # Options: (0.04,0.06,0.09, 0.65). 

    # These values have to be computed the first time I run this notebook.
    # NOTE: The values of "Average_NIRAbsMag_TBmax" and "error_Average_NIRAbsMag_TBmax" do NOT 
    # depend on the value of "vpecFix".

    if zcmbUpperLim == 0.04: 

        # ------------------------------
        #   Y band | vpecFix = 150 km/s | 0 < z < 0.04
        Average_NIRAbsMag_TBmax = -18.127153489;  # 2018-07-12; 21:02 hrs.
        error_Average_NIRAbsMag_TBmax = 0.160051080419;

    elif zcmbUpperLim == 0.06:
        # Using Foley + Cepheid + Special cases's zcmb
        Average_NIRAbsMag_TBmax = -18.2640428698;   
        error_Average_NIRAbsMag_TBmax = 0.145468431004; 


    elif zcmbUpperLim == 0.09: 
        # Using Foley + Cepheid + Special cases's zcmb
        Average_NIRAbsMag_TBmax = nan ;   
        error_Average_NIRAbsMag_TBmax = nan ; #


    # Ignore data with a chi^2_dof larger than a given threshold:
    # Use the values ( chi2_dof_Max = 1e6; chi2_dof_Max_Label = 'chi_1e6') 
    # the first time I run the notebook.
    # chi2_dof_Max = 3.5; chi2_dof_Max_Label = 'chi3_5'
    chi2_dof_Max = 3; chi2_dof_Max_Label = 'chi3'

    l_kern = 7.9874; 

    # PHASE RANGE IN DAYS OF TEMPLATE TO USE TO COMPUTE THE DISTANCE MODULUS
    PhaseMinTemp = -8 # -8 days
    PhaseMaxTemp = 30 # 30, 41, days

    #-- EBV cutoff
    EBVhostMin = -0.4 # -0.4 # host galaxy
    EBVhostMax = 0.4 # 0.4 # host galaxy. 
    EBVMWLim = 1 # Milky-Way galaxy

    #-- dm15 cutoff
    dm15LowerLim = 0.8 # I assume 0.8.
    dm15UpperLim = 1.6
    
    # - Smoothed Moving windows average to construct the template
    # If TempType = 'MWA' then specify the template file to use
    MWATempTypeFile = 'TempWeightedSmooth_Box7_Step05_Window21_Poly3.dat'

###########################################################

if BandName == 'H': 

    # Redshift cutoff.
    # In the low-z paper we use zcmbUpperLim = 0.04.
    zcmbUpperLim = 0.04      # Options: (0.04,0.06,0.09, 0.65). 

    # These values have to be computed the first time I run this notebook.
    # NOTE: The values of "Average_NIRAbsMag_TBmax" and "error_Average_NIRAbsMag_TBmax" do NOT 
    # depend on the value of "vpecFix".

    # The value of 'IntrinsicDisp_GP' comes from the Gaussian-Process Hubble diagram

    if zcmbUpperLim == 0.04: 
        
        # ------------------------------
        #   H band | vpecFix = 150 km/s | 0 < z < 0.04
        # Average_NIRAbsMag_TBmax = -18.1321913853;  # 2018-04-26; 02:22 hrs.
        # error_Average_NIRAbsMag_TBmax = 0.17494126888;
        
        # ------------------------------
        #   H band | vpecFix = 150 km/s | 0 < z < 0.04
        Average_NIRAbsMag_TBmax = -18.1847957204;  # 2018-07-12; 22:48 hrs.
        error_Average_NIRAbsMag_TBmax = 0.163480843711;


    elif zcmbUpperLim == 0.06: 
        # Using Foley + Cepheid + Special cases's zcmb: 
        Average_NIRAbsMag_TBmax = -18.1365343086;   error_Average_NIRAbsMag_TBmax = 0.166013465848 ;   
         

    elif zcmbUpperLim == 0.09: 
        # Using Foley + Cepheid + Special cases's zcmb
        Average_NIRAbsMag_TBmax = nan ;   error_Average_NIRAbsMag_TBmax = nan ; #


    # Ignore data with a chi^2_dof larger than a given threshold:
    # Use the values ( chi2_dof_Max = 1e6; chi2_dof_Max_Label = 'chi_1e6') 
    # the first time I run the notebook.
    chi2_dof_Max = 1e6; chi2_dof_Max_Label = 'chi_1e6'
    # chi2_dof_Max = 3.5; chi2_dof_Max_Label = 'chi3_5'

    l_kern = 9.9966; # From the normalized template (with peculiar velocity 0 km/s).

    # PHASE RANGE IN DAYS OF TEMPLATE TO USE TO COMPUTE THE DISTANCE MODULUS
    PhaseMinTemp = -8 # -8 days
    PhaseMaxTemp = 30 # 30, 41, days

    # Not in use yet. AbsMag_NIRmax= -18.4670;  error_AbsMag_NIRmax=0.02974; phase_NIRmax = -4.5; 

    #-- EBV cutoff
    EBVhostMin = -0.4 # -0.4 # host galaxy
    EBVhostMax = 0.4 # 0.4 # host galaxy. 
    EBVMWLim = 1 # Milky-Way galaxy

    #-- dm15 cutoff
    dm15LowerLim = 0.8 # I assume 0.78.
    dm15UpperLim = 1.6
    
    # - Smoothed Moving windows average to construct the template
    # If TempType = 'MWA' then specify the template file to use
    MWATempTypeFile = 'TempWeightedSmooth_Box7_Step05_Window21_Poly3.dat'

###########################################################
 
if BandName == 'K': 

    # Redshift cutoff.
    # In the low-z paper we use zcmbUpperLim = 0.04.
    zcmbUpperLim = 0.04      # Options: (0.04,0.06,0.09, 0.65). 

    # These values have to be computed the first time I run this notebook.
    # NOTE: The values of "Average_NIRAbsMag_TBmax" and "error_Average_NIRAbsMag_TBmax" do NOT 
    # depend on the value of "vpecFix".

    if zcmbUpperLim == 0.04: # 
        
        # ------------------------------
        #   K band | vpecFix = 150 km/s | 0 < z < 0.04
        # Average_NIRAbsMag_TBmax = -18.431089343;  # 2018-04-26; 13:09 hrs.
        # error_Average_NIRAbsMag_TBmax = 0.214342169223;
        
        # ------------------------------
        #   K band | vpecFix = 150 km/s | 0 < z < 0.04
        Average_NIRAbsMag_TBmax = -18.3562703093;  # 2018-07-12; 23:02 hrs.
        error_Average_NIRAbsMag_TBmax = 0.209567671681;


    elif zcmbUpperLim == 0.06:
        # Using Foley + Cepheid + Special cases's zcmb
        Average_NIRAbsMag_TBmax = nan ;   
        error_Average_NIRAbsMag_TBmax = nan ; #


    elif zcmbUpperLim == 0.09: 
        # Using Foley + Cepheid + Special cases's zcmb
        Average_NIRAbsMag_TBmax = nan ;   
        error_Average_NIRAbsMag_TBmax = nan ; #

    # Ignore data with a chi^2_dof larger than a given threshold:
    # Use the values ( chi2_dof_Max = 1e6; chi2_dof_Max_Label = 'chi_1e6') 
    # the first time I run the notebook.
    # chi2_dof_Max = 1e6; chi2_dof_Max_Label = 'chi_1e6'
    chi2_dof_Max = 4; chi2_dof_Max_Label = 'chi4'

    l_kern = 8.2101; 

    # PHASE RANGE IN DAYS OF TEMPLATE TO USE TO COMPUTE THE DISTANCE MODULUS
    PhaseMinTemp = -8 # -8 days
    PhaseMaxTemp = 30 # 30, 41, days

    # Not in use yet. AbsMag_NIRmax= -18.2861;  error_AbsMag_NIRmax=0.03908; phase_NIRmax = -4; 

    #-- EBV cutoff
    EBVhostMin = -0.4 # -0.4 # host galaxy
    EBVhostMax = 0.4 # 0.4 # host galaxy. 
    EBVMWLim = 1 # Milky-Way galaxy

    #-- dm15 cutoff
    dm15LowerLim = 0.8 # I assume 0.78.
    dm15UpperLim = 1.6
    
    # - Smoothed Moving windows average to construct the template
    # If TempType = 'MWA' then specify the template file to use
    MWATempTypeFile = 'TempWeightedSmooth_Box7_Step05_Window21_Poly3.dat'
    
#########################################################################

# Warning in case that the mean abs mag was not defined:
if Average_NIRAbsMag_TBmax == None and  error_Average_NIRAbsMag_TBmax == None:
    print "# ERROR: The mean abs magnitude and its std dev have to be defined."

#########################################################################

# Text in the notes about the origin of (Average_NIRAbsMag_TBmax, error_Average_NIRAbsMag_TBmax)
if AbsMagFromHisto == True:
    Notes1 = 'Determined from the histogram of {appMagTmax_s - mu_LCDM_s}' 
elif AbsMagFromHisto == False:
    Notes1 = 'Just a random guess about these values. It doesnt matter the value of this guess \
because this numbers are to determine the distance modulus but in this first step I m computing \
the appMag_TBmax only.'  

#------------------

# About 'l_kern': Lenght scale 'l' of the kernel in the covariance matrix for the residual.
# this value is ignored in case of 'CovMat_MeanMu = False'
# The values I obtained when using the global marginal likehood function to compute 
# the hyperparameter 'l' using all the LCs together.

#==============================================================

#             Method to determine the distance modulus
# (see description below)

Method = 7 # Options  = (1,2,3,4,5,6,7). In the paper I use method 7, before I was using 5.
# The best methods are 5 (unnormalized template) and 7 (normalized template).

# COVARIANCE MATRICES
# It is not necessary to modify the following 2 lines unless some very particular computation is needed.
# Using the covariance matrix of the photometry to determine the -mean- and -uncertainty-
# of the distance modulus?:
if Method==1 or Method==5 or Method==6 or Method==7: CovMat_MeanMu = False; CovMat_ErrorMu = True
elif Method==2 or Method==3 or Method==4: CovMat_MeanMu = True; CovMat_ErrorMu = True
    
# CovMat_MeanMu = False # (True, False). # In the paper I use 'False'
# CovMat_ErrorMu = False # (True, False). # In the paper I use 'True'

# Use the peculiar velocity covariance square matrix to determine the distance modulus?: 
#OLD.  Use_CovMat_PecVel = True # (True, False).
# NOTE: When "Use_CovMat_PecVel == True" then in the total covariance matrix it is used the -uncertainty- 
# in the mean template instead of the -population standard deviation- of the template.

#-------------

#-- Minimal number of data in LC:
MinNumOfDataInLC = 3

# Print the (RMS, WRMS, sigma_int) text in the residual plot?:
WRMS_label = True # Print the WRMS of the total sample (or total+subsamples)?
RMS_simple = True # or print instead the simple RMS only of total and subsamples?
WRMS_subsamples = True # Print the RMS, WRMS of each subsample?

#=================================================================
#( Fixed values usually) 

# --- (FIX) Residual cutoff ---
# residualMax = 0.7; residualMax_Label = 'resid07'
# residualMax = 0.8; residualMax_Label = 'resid08'
# residualMax = 0.9; residualMax_Label = 'resid09'
# residualMax = 1; residualMax_Label = 'resid1'
# residualMax = 3; residualMax_Label = 'resid3'
# residualMax = 5; residualMax_Label = 'resid5' 
residualMax = 20; residualMax_Label = 'resid20' 

#---- (FIX) Limits in the plots for the Hubble diagram and residual  ----

# For the Hubble diagram:
xlimPlots = 0.0015, zcmbUpperLim+0.003 # For low-z only
ylimPlots = 29.8, 38 # For low-z only

# For the Hubble residual
ylimPlots_residual = -1, 1

#----------------

# Plotting range of the fitted LC figures.
x_RangePlots = -10, 60;

# In the plots of the individual LC, print the chi2_dof value?:
Chi2dofPrint = True 

#--------------------------------------------

#---- (FIX) Filter system ----
FilterSyst = 'Std_filters/'
# FilterSyst = 'CSP_filters/'

In [4]:
# Description of the methods

""" 
- Method 1: 
	- No use of CovMatrix to determine the best estimate of distance mu (but still optional using 
    "CovMat_use == True" [default: "CovMat_use == False" ]),  BUT using it (i.e., "CovMat_use == True") 
    to determine its uncertainty [default: "CovMat_use == True" ].
	- using the uncertainty in the mean template instead of the population standard deviation
	- adding the uncertainty in the distance modulus due to the peculiar velocity uncertainty in the total 
    covariance matrix -before- to compute the distance-modulus uncertainty.
	- CovMat_MeanMu == False
	- CovMat_ErrorMu == True
	- Template = np.genfromtxt('Template_phase_mu_stdError_FromR.dat')
	- CovMatrix_Mu = CovMatResidualMu + Cov_appmag # Total covariance matrix -before- computing
	- CovMatrix_ErrorMu = CovMatCorrData_4ErrorMu + Cov_appmag + Cov_pecvel # Total covariance matrix -before- 
    computing
	- sigma2_mu = (1/Denominator_ErrorMu) # Variance mu
	

- Method 2: 
	- using the uncertainty in the mean template instead of the population standard deviation
	- mean mu: using the CovMat_MeanMu == True and ignoring Cov_pecvel to compute it.
	- error mu: adding in quadratures the uncertainty in the distance modulus due to the peculiar velocity 
    uncertainty with the uncertainty in the distance modulus from fitting the template -after- computing 
    the distance modulus
    - Apparently, this method gives exactly the same results than Method 3
    - CovMat_MeanMu == True
	- CovMat_ErrorMu == True
	- Template = np.genfromtxt('Template_phase_mu_stdError_FromR.dat')
	- CovMatrix_Mu = CovMatResidualMu + Cov_appmag # Total covariance matrix -before- computing
	- CovMatrix_ErrorMu = CovMatCorrData_4ErrorMu + Cov_appmag # Total covariance matrix -before- computing
	- sigma2_mu = (1/Denominator_ErrorMu) +  sigma2_pec(zcmbInt, err_zcmb, vpecFix) # Variance mu


- Method 3: 
	- using the uncertainty in the mean template instead of the population standard deviation
	- adding the uncertainty in the distance modulus due to the peculiar velocity uncertainty in the total 
    covariance matrix -before- to compute the mean distance modulus and its uncertainty.
    - Apparently, this method gives exactly the same results than Method 2
	- CovMat_MeanMu == True
	- CovMat_ErrorMu == True	
	- Template = np.genfromtxt('Template_phase_mu_stdError_FromR.dat')
	- CovMatrix_Mu = CovMatResidualMu + Cov_appmag + Cov_pecvel # Total covariance matrix -before- computing
	- CovMatrix_ErrorMu = CovMatCorrData_4ErrorMu + Cov_appmag + Cov_pecvel # Total covariance matrix -before- 
    computing
	- sigma2_mu = (1/Denominator_ErrorMu) # Variance mu
	
	
- Method 4: 
	- using the population standard deviation of the template
	- NOT including the uncertainty in the distance modulus due to the peculiar velocity during the fitting.
	= It produces very small error bars for the distance-modulus uncertainty
	- CovMat_MeanMu == True
	- CovMat_ErrorMu == True	
	- Template = np.genfromtxt('Template_phase_mu_tau_FromR.dat')
	- CovMatrix_Mu = CovMatResidualMu + Cov_appmag # Total covariance matrix -before- computing
	- CovMatrix_ErrorMu = CovMatCorrData_4ErrorMu + Cov_appmag # Total covariance matrix -before- computing
	- sigma2_mu = (1/Denominator_ErrorMu) # Variance mu
    
- Method 5: 
    - Equal to the method 1 with the difference of using the -population standard deviation- of the template 
    instead of its uncertainty. The idea of this is that during the fitting of the template to the app mag data 
    is that it gives more weight to the data around the first NIR peak than the data in the 2nd peak because the 
    population standard deviation of the template is larger around the 2nd peak, so that my fitting is not 
    screwed up by the fact that the 2nd peak seems to have an independent variability between the SNe, then 
    biasing my distance mu estimations.
	- CovMat_MeanMu == False
	- CovMat_ErrorMu == True
	- Template = np.genfromtxt('Template_phase_mu_tau_FromR.dat')
	- CovMatrix_Mu = CovMatResidualMu + Cov_appmag # Total covariance matrix -before- computing
	- CovMatrix_ErrorMu = CovMatCorrData_4ErrorMu + Cov_appmag + Cov_pecvel # Total covariance matrix -before- computing
	- sigma2_mu = (1/Denominator_ErrorMu) # Variance mu
	
    
- Method 6:
    - So far, equal to method 5 but now normalizing the template, then fit it to determine the apparent 
    magnitude at T_Bmax, then subtract the absolute magnitude at T_Bmax to determine the photometric
    distance modulus.
    - I take the regular template constructed in the absolute magnitude frame then I normalize it here in this 
    notebook, so that 
    at T_Bmax I set the magnitude to be equal zero.
    
- Method 7:
    - I used a normalized template that was normalized by construction in the hierarchical-Bayesian-R script.
    - The rest, same than Method 6.
    - The notebook has to be run twice. The first run is to determine the NIR apparent magnitude at t_Bmax only
      by fitting the template to the apparent-magnitude light curves, then using the relation, 
      AbsMag_s = appMag_TBmax - mu_LCDM, it is determined the NIR absolute magnitude at t_Bmax, AbsMag_s, for
      each SN. In this run it is created a value of the distance modulus but it is not relevant/reliable. 
      After the main loop, there is a section to determine <Average_NIRAbsMag_TBmax> from the collection of 
      {Average_NIRAbsMag_TBmax}
      
      In the second run is computed final distance modulus from: 
              mu_photo_Analytic = appMag_TBmax - <Average_NIRAbsMag_TBmax>
      In the second run the following quantities are again re-computed like in the first run: 
      (appMag_TBmax, error_appMagTBmax, mu_LCDM, sigma_muLCDM_vPec, AbsMagTBmax, error_AbsMagTBmax, chi2_dof),
      so, their values are -exactly- the same as those obtained during the first run. In the second run, the only 
      new quantities compared with run 1 are: (mu_photo_Analytic, stdDev_mu, mu_resid)

"""
0

0

---------------

# Automatic

In [5]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import quad as intquad
from scipy.optimize import fmin as simplex

import os
import glob # To read the name of the files in a given directory

4+5

9

#### Get the name of this ipython notebook
To print it in the output text files as reference.

In [6]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "NotebookName = " + "'"+thename+".ipynb"+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [7]:
print(NotebookName)
# 11_DistanceMu_HubbleDiagram_v2_17.ipynb

11_DistanceMu_HubbleDiagram.ipynb


In [8]:
# Get the current date and time
import datetime 

# Read the time and date now
now = datetime.datetime.now()

In [9]:
#- Function to convert from index to days (phase)

if TempType=='MWA': shiftNum = 24
else: shiftNum = 71
    
#- Function to convert from index to days (phase).
def index2day(index):
    day = (index-shiftNum)/2.
    return day

#- Function to convert from days (phase) to index.
def day2index(day):
    index = 2*day + shiftNum
    return index

print 'Testing the functions (%s):'%TempType, index2day(105), ',', day2index(-11)

#--------------------------------
# Converting phases to indices

index_LowerPhase = day2index(PhaseMinTemp)
index_UpperPhase = day2index(PhaseMaxTemp)

LowerLabel = str(PhaseMinTemp)
UpperLabel = str(PhaseMaxTemp)

#--------------------------------
# OLD. Testing the functions (GP): 42 , 155
# Testing the functions (MWA): 40.5 , 2
# Testing the functions (FlatPrior): 17.0 , 49

Testing the functions (FlatPrior): 17.0 , 49


In [10]:
# Plotting Settings:

if BandName == 'J':
    # For overplotting the template to the fitted LC.
    # MinMagTempPlot = minimum value in the template plot
    if Method==7: MinMagTempPlot = 3.2
    else: MinMagTempPlot = -15
    
elif BandName == 'Y':
    # For overplotting the template to the fitted LC.
    # MinMagTempPlot = minimum value in the template plot
    if Method==7: MinMagTempPlot = 2
    else: MinMagTempPlot = -15.5
    
elif BandName == 'H':
    # For overplotting the template to the fitted LC.
    # MinMagTempPlot = minimum value in the template plot
    if Method==7: MinMagTempPlot = 2
    else: MinMagTempPlot = -15.5
    
elif BandName == 'K':
    # For overplotting the template to the fitted LC.
    # MinMagTempPlot = minimum value in the template plot
    if Method==7: MinMagTempPlot = 2
    else: MinMagTempPlot = -15.5

#### Defining and creating directories

In [11]:
# Defining the directories

DirMain_1 = '/Users/arturo/Dropbox/Research/Articulos/10_AndyKaisey/\
10Compute/TheTemplates/' 

DirMain = (DirMain_1+BandName+'_band/'+FilterSyst)  

DirHubbleDiag = DirMain+'4_HubbleDiagram_%s/'%(TempType)

if CovMat_MeanMu == True:
    # OLD. CovMatLabel_MeanMu = 'lkern%s'%(l_kern)+'_CovMat'
    CovMatLabel_MeanMu = 'CovMatMu'
elif CovMat_MeanMu == False:
    CovMatLabel_MeanMu = 'NoCovMatMu'
    
if CovMat_ErrorMu == True:
    CovMatLabel_ErrorMu = 'CovMatErrorMu'
elif CovMat_ErrorMu == False:
    CovMatLabel_ErrorMu = 'NoCovMatErrorMu'
    
# Old.
# DirSaveOutput = (DirHubbleDiag + KindOfData4HD +'/Templ_'+KindOfTemp+'_'+
#                  KindOfTempSubgroup+'/Phase'+LowerLabel+'_'+UpperLabel+'_'+residualMax_Label+'_'+
#                  chi2_dof_Max_Label+'_EBVh%r'%EBVhostMax+'_Method%r'%Method+
#                  '_'+CovMatLabel_MeanMu+'_'+CovMatLabel_ErrorMu+
#                  '_MinData%r'%MinNumOfDataInLC+'_vpec%r'%vpecFix+'/')

DirSaveOutput = (DirHubbleDiag + KindOfData4HD +'/Templ_'+KindOfTemp+'_'+
                 KindOfTempSubgroup+'/Phase'+LowerLabel+'_'+UpperLabel+'_'+residualMax_Label+'_'+
                 chi2_dof_Max_Label+'_EBVh%r'%EBVhostMax+'_Method%r'%Method+
                 '_MinData%r'%MinNumOfDataInLC+'_vpec%r'%vpecFix+'/plots_HD/')

# old. DirAppMag = DirMain+'1_AllData_InitialFit/AppMag/'
DirAppMag = DirMain+'1_AllData_InitialFit/AbsMag/AllSamples/'

# Dir template:
DirTemplate = DirMain+'3_Template_%s/'%(TempType)+KindOfTemp+'_vpec_0/'+KindOfTempSubgroup+'/'

#- Force the creation of the directory to save the outputs.
#- "If the subdirectory does not exist then create it"
import os # To use command line like instructions
if not os.path.exists(DirSaveOutput): os.makedirs(DirSaveOutput)

# Dir to save the plots of each fitted SNe
if NotebookToPlotOnly == False:
    if not os.path.exists(DirSaveOutput+'Plot_Fits/'): os.makedirs(DirSaveOutput+'Plot_Fits/')
    if not os.path.exists(DirSaveOutput+'Plot_Fits/AfterCutoffs_z001/'): os.makedirs(DirSaveOutput+'Plot_Fits/AfterCutoffs_z001/')
    if not os.path.exists(DirSaveOutput+'Plot_Fits/AfterCutoffs_z0/'): os.makedirs(DirSaveOutput+'Plot_Fits/AfterCutoffs_z0/')
    if not os.path.exists(DirSaveOutput+'Plot_Fits/OutCutoffs/'): os.makedirs(DirSaveOutput+'Plot_Fits/OutCutoffs/')

# Visualizing the directories
print 'Dir to save the outputs:'
print DirSaveOutput
print ' '

print 'Template to be used to fit the LC data:'
print DirTemplate
print ' '

print 'Dir where the LCs in APPARENT mag are located:'
print DirAppMag
print ' '

Dir to save the outputs:
/Users/arturo/Dropbox/Research/Articulos/10_AndyKaisey/10Compute/TheTemplates/Y_band/Std_filters/4_HubbleDiagram_FlatPrior/AllSamples/Templ_AllSamples_z_gr_0/Phase-8_30_resid20_chi3_EBVh0.4_Method7_MinData3_vpec150/plots_HD/
 
Template to be used to fit the LC data:
/Users/arturo/Dropbox/Research/Articulos/10_AndyKaisey/10Compute/TheTemplates/Y_band/Std_filters/3_Template_FlatPrior/AllSamples_vpec_0/z_gr_0/
 
Dir where the LCs in APPARENT mag are located:
/Users/arturo/Dropbox/Research/Articulos/10_AndyKaisey/10Compute/TheTemplates/Y_band/Std_filters/1_AllData_InitialFit/AbsMag/AllSamples/
 


In [12]:
if NotebookToPlotOnly == False:
    
    import os # To use command line like instructions
    import glob # To read the files in my directory

    # Change the working directory where the data files are located
    os.chdir(DirAppMag)

    #--- Reading the data files in the app mag folder 
    if KindOfData4HD == 'CfA':
        list_SNe2 = glob.glob('*'+'CfA_'+BandName+'.txt')
    elif KindOfData4HD == 'CSP':
        list_SNe2 = glob.glob('*'+'CSP_'+BandName+'.txt')
    elif KindOfData4HD == 'Others':
        list_SNe2 = glob.glob('*'+'Others_'+BandName+'.txt')
    elif KindOfData4HD == 'AllSamples':
        list_SNe2 = glob.glob('*'+BandName+'.txt')

    print '# Number of -%s- SNe in the APPARENT mag list (%s):'%(
        KindOfData4HD,TempType), len(list_SNe2)

    # Number of SNe in APPARENT mag list: 132
    # Number of -AllSamples- SNe in the APPARENT mag list (MWA): 142
    # Number of -AllSamples- SNe in the APPARENT mag list (FlatPrior): 174

In [13]:
if NotebookToPlotOnly == False:

    #     CREATE &/or READ THE NAME OF THE SNe FROM THE TEXT FILE

    # Change the working directory where the data files are located
    os.chdir(DirSaveOutput)
    # Reading the data files in that folder 
    Textfiles = glob.glob('ListSNe_AppMag*.txt')

    # Reset the variable
    list_SNe = np.array([0])

    if 'ListSNe_AppMag_Notes_.txt' in Textfiles:
        list_SNe = np.genfromtxt(DirSaveOutput+'ListSNe_AppMag_Notes_.txt', dtype=['S33', int])
        print '# Reading the existing < ListSNe_AppMag_Notes_.txt > file.'
        print "# %s SNe file names in this list and uploaded to RAM memory."%len(list_SNe)

    elif 'ListSNe_AppMag_.txt' in Textfiles:
        list_SNe = np.genfromtxt(DirSaveOutput+'ListSNe_AppMag_.txt', dtype=['S33', int])
        print '# Reading the existing < ListSNe_AppMag_.txt > file.'
        print "# %s SNe file names in this list and uploaded to RAM memory."%len(list_SNe)

    #--------------------------------------------------

    # if ListSNe_AppMag_.txt doesn't exist, then create it and read it.

    else: 
        # Create a list text file with the name of the SNe of the apparent mag LCs.
        list_SNe_file = open(DirSaveOutput+'ListSNe_AppMag_.txt', 'w')

        list_SNe_file.write('# SN list of LCs to be used to \
construct the Hubble diagram \n')
        list_SNe_file.write('# I m NOT applying any quality cutoff yet: These \
SNe are ALL those located in: \n')
        list_SNe_file.write('# %s \n'%DirAppMag)

        #------
        now = datetime.datetime.now() # Read the time and date right now
        text_timenow = now.strftime("%Y-%m-%d (yyyy-mm-dd)")
        text_Date   = '# On date: %s \n'%text_timenow
        text_Author = '# Data table created by: Arturo Avelino \n'
        text_script = '# Script used: %s \n'%NotebookName
        text_line = '#'+'-'*50 + '\n'

        list_SNe_file.write(text_line); 
        list_SNe_file.write(text_Author); list_SNe_file.write(text_Date); list_SNe_file.write(text_script);
        list_SNe_file.write(text_line);
        #------

        # Upload the list of repeated SNe between CfA and CSP
        DirRepeatedSNe = '/Users/arturo/Dropbox/Research/SoftwareResearch/\
Snoopy/AndyLCComp_2018_02/MyNotes/'
        RepeatedSNeList = np.genfromtxt(DirRepeatedSNe+'RepeatedSNe_between_CfA_CSP.txt',
                                       dtype=['S10', int])

        # Upload the list of SNe to discard from the Hubble diagrams
        DirSNeWithIssues = '/Users/arturo/Dropbox/Research/SoftwareResearch/\
Snoopy/AndyLCComp_2018_02/MyNotes/'
        SNeWithIssuesList = np.genfromtxt(DirRepeatedSNe+'SNeWithIssues.txt',
                                       dtype=['S30', 'S150'])

        for name in list_SNe2:
            snname_int =    '%s'%name[0:8]
            subsample_int = '%s'%name[-9:-6]
            # print snname_int, subsample_int

            if snname_int not in SNeWithIssuesList['f0']:
                if snname_int not in RepeatedSNeList['f0']:
                    # SNe to be NO commented:
                    list_SNe_file.write('%-32s  0 \n'%name)

                # Comment automatically the repeated SNe.
                elif snname_int in RepeatedSNeList['f0']:
                    for i1 in range(len(RepeatedSNeList['f0'])):
                        if snname_int == RepeatedSNeList['f0'][i1]:
                            if RepeatedSNeList['f1'][i1] == 1 and subsample_int =='CSP':
                                list_SNe_file.write('%-32s    1 ## Repeated. CfA selected. \n'%name)
                            elif RepeatedSNeList['f1'][i1] == 2 and subsample_int =='CfA':
                                list_SNe_file.write('%-32s    1 ## Repeated. CSP selected. \n'%name)
                            else:
                                list_SNe_file.write('%-32s  0 \n'%name)

            elif snname_int in SNeWithIssuesList['f0']:
                for i2 in range(len(SNeWithIssuesList['f0'])):
                    if snname_int == SNeWithIssuesList['f0'][i2]:
                        list_SNe_file.write('%-32s    1 ## %s \n'%(
                            name, SNeWithIssuesList['f1'][i2]))

        list_SNe_file.write(text_line)
        list_SNe_file.write('# %s SNe in total in this list. \n'%(len(list_SNe2)))
        list_SNe_file.close()

        #--------------------------------------

        # Read the list I've just created:

        list_SNe = np.genfromtxt(DirSaveOutput+'ListSNe_AppMag_.txt', dtype=['S33', int])
        print '# Reading: < ListSNe_AppMag_.txt >'

    print '# %s SNe in total in this list.'%len(list_SNe)
    print '# %s SNe masked in the list.'%sum(list_SNe['f1'])

In [14]:
# Reading: <ListSNe_AppMag_.txt>
# Number of SNe in the list = 163
# Number of masked SNe in the list:  0

### Interpolating the template

In [15]:
if NotebookToPlotOnly == False:

    from scipy.interpolate import interp1d # To interpolate

    if TempType == 'MWA':
        Template = np.genfromtxt(DirTemplate+MWATempTypeFile) 
        # Index of max abs mag in template. It doesn't need to be very precise; 
        # it is just to define the plot limit in y-axis
        indexMaxTempl = 17 # --> phase = -3.5
        indexLowerPhase = index_LowerPhase-1
        indexUpperPhase = index_UpperPhase
        #-- Interpolating the template
        MTemplInt       = interp1d(Template[indexLowerPhase:indexUpperPhase,0] , 
                                   Template[indexLowerPhase:indexUpperPhase,1] , 
                                   kind='linear')
        error_MTemplInt = interp1d(Template[indexLowerPhase:indexUpperPhase,0] , 
                                   np.sqrt(Template[indexLowerPhase:indexUpperPhase,3]) , 
                                   kind='linear')

    else:
        #OLD. if Use_CovMat_PecVel == True: Template = np.genfromtxt(DirTemplate + 
        # 'Template_phase_mu_stdError_FromR.dat')
        #OLD. else: Template = np.genfromtxt(DirTemplate+'Template_phase_mu_tau_FromR.dat') 

        if Method==1 or Method==2 or Method==3: TemplateFile = 'Template_phase_mu_stdError_FromR.dat'
        elif Method==4 or Method==5 or Method==6: TemplateFile = 'Template_phase_mu_tau_FromR.dat'
        elif Method==7: TemplateFile = 'Template_phase_mu_tau_FromR_Norma.dat'

        Template = np.genfromtxt(DirTemplate+TemplateFile)

        # Index of max abs mag in template. It doesn't need to be very precise, 
        # it is just to define the plot limit in y-axis.
        indexMaxTempl = 64 # --> phase = -3.5

        #--- Interpolating the template ---

        indexLowerPhase = index_LowerPhase-1
        indexUpperPhase = index_UpperPhase


        if Method==6: # Normalizing the template
            Average_NIRAbsMag_TBmax = Template[(day2index(0)-1),1] # Abs mag at T_Bmax in the template.
            # old. Average_NIRAbsMag_TBmax_stdDev = Template[(day2index(0)-1),2] # Population standard deviation of the abs mag at T_Bmax in the template.

            MTemplInt       = interp1d(Template[indexLowerPhase:indexUpperPhase,0] , 
                                       Template[indexLowerPhase:indexUpperPhase,1]-Average_NIRAbsMag_TBmax , kind='linear') 

            TemplateError = np.genfromtxt(DirTemplate+'Template_phase_mu_stdError_FromR.dat')
            error_Average_NIRAbsMag_TBmax = TemplateError[(day2index(0)-1),2] # Uncertainty int the abs mag at T_Bmax in the template.

        else:
            MTemplInt       = interp1d(Template[indexLowerPhase:indexUpperPhase,0] , 
                                       Template[indexLowerPhase:indexUpperPhase,1] , kind='linear')

        error_MTemplInt = interp1d(Template[indexLowerPhase:indexUpperPhase,0] , 
                                   Template[indexLowerPhase:indexUpperPhase,2] , kind='linear')

    print '# Test (%s). Value of the template at phase = -1.2: (%.6f, %.6f). %s band'%(
        TempType, MTemplInt(-1.2), error_MTemplInt(-1.2), BandName)

    # OLD. Test (FlatPrior). Value of the template at phase = -1.2:  -0.0739400198048 0.0268277891928 # J band
    # Test (FlatPrior). Value of the template at phase = -1.2: (-0.066778, 0.016821). J band

In [16]:
if NotebookToPlotOnly == False:
    
    # Checking the range of interpolation
    Template[indexLowerPhase:indexUpperPhase,0]

In [17]:
if NotebookToPlotOnly == False:
    # Checking the range of interpolation
    print Template[indexLowerPhase][0], Template[indexUpperPhase-1][0]
    # -8.0 31.0

In [18]:
if NotebookToPlotOnly == False:
    
    # Checking the interpolation at T_Bmax
    print MTemplInt(0)
    # 0.0
    # -0.000032300235868e-05
    # -8.17052766133e-06

In [19]:
if NotebookToPlotOnly == False:

    if Method==6 or Method==7:
        print 'Average_NIRAbsMag_TBmax = ', Average_NIRAbsMag_TBmax
        print 'error_Average_NIRAbsMag_TBmax', error_Average_NIRAbsMag_TBmax

# J band:
# Average_NIRAbsMag_TBmax =  -18.3805198211
# error_Average_NIRAbsMag_TBmax 0.0247540912503

## $\mu_{\rm \Lambda CDM}$

In [20]:
# Inverse of the dimensionless Hubble parameter
def InvEHubblePar(z, OmM, wde):
    "Dimensionless Hubble parameter"
    InvEHubbleParInt = 1.0/(np.sqrt(OmM*(1.0+z)**3.0 + (1.0-OmM)*(1.+z)**(3.*(1.+wde))))
    return InvEHubbleParInt

# ---- The luminosity distance ----
def LumDistanceVec(z, OmM, wde, Ho):
    "Luminosity distance"
    LumDistanceVecInt = 0.
    LumDistanceVecInt = c*(1.+z)*intquad(InvEHubblePar, 0., z, args=(OmM, wde))[0]/Ho 
    return LumDistanceVecInt

# ---- Distance modulus scalar ----
def DistanceMu(z, OmM, wde, Ho):
    "Distance modulus"     
    DistanceMuInt = 5.0*np.log10(LumDistanceVec(z, OmM, wde, Ho)) + 25.0
    return DistanceMuInt

# ---- Distance modulus Vector ----
def DistanceMuVector(z, OmM, wde, Ho):
    "Distance modulus"     
    DistanceMuInt= []
    for i in range(len(z)):
        DistanceMuInt += [5.0*np.log10(LumDistanceVec(z[i], OmM, wde, Ho)) + 25.0] 
    return DistanceMuInt

#--------------------------------------------------

ztest1 = 0.01

print 'Checking that the functions work well:', DistanceMu(ztest1, OmMFix, wFix, HoFix)
# Checking that the functions work well: 33.1141460988 # Ho=72
# Checking that the functions work well: 33.0773926577 # Ho=73.24

Checking that the functions work well: 33.0773926577


In [21]:
# Creation of an array of redshift vs theoretical distance modulus (for Pete Challis)
""" 
z1 = np.linspace(0.01, 0.7, 501)
DistMu_array = DistanceMuVector(z1, OmMFix, wFix, HoFix)

DirSaveOutput = '/Users/arturo/Dropbox/Research/Articulos/0_LCDM/'
textfile = open(DirSaveOutput+'DistanceModulus.txt', 'w')
textfile.write('# Cosmology: flat Universe, Om_matter = %r, Om_Lambda = %r, Ho = %r km/s/Mpc \n'%(OmMFix, 1-OmMFix, HoFix))
textfile.write('# z        Distance modulus  \n')

for i in range(len(z1)):
    textfile.write('%.6f   %.6f \n'%(z1[i], DistMu_array[i]))
    
textfile.close()
"""
0

0

### Uncertainty in distance modulus due to the peculiar-velocity uncertainty

In [22]:
# sigma^2_mu from the peculiar velocity uncertainty
# This function is used to determine in the sections "Intrinsic dispersion" and "Optical RMS", to
# determine the intrinsic dispersion.

def sigma2_pec(zcmb, err_zcmb, vpec):
    sigma2_pecInt = ((5/(zcmb*np.log(10)))*np.sqrt((vpec/c)**2 + err_zcmb**2))**2
    return sigma2_pecInt

# Test
sigma2_pec(0.0109942726, 0.0010420420, 150)
# 0.052125037354329877

0.052125037354329877

#### Function to identify string or number

In [23]:
# Function to identify if a string is an integer number or a letter.
# This will be used in the dictionary construction to properly read some SN names.

def is_number(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

# Tests
print is_number('5'), is_number('e')
# True False

True False


### Cepheid distances

In [24]:
DirSNeWithCepheid = '/Users/arturo/Dropbox/Research/SoftwareResearch/Snoopy/AndyLCComp_2018_02/MyNotes/'

# From the Cepheid SNe list the only part that I use in the entire
# notebook is the first column, i.e., the SN Name column.
ListSNeCepheid = np.genfromtxt(DirSNeWithCepheid+'SNeWithCepheidDistances.txt', dtype=['S10', 
                                                float,float,float,float,float,float])

print "# %s SNe with Cepheid distances in Andy's compilation. "%len(ListSNeCepheid['f0'])
ListSNeCepheid['f0']

# 13 SNe with Cepheid distances in Andy's compilation. 


array(['sn1981B', 'sn1998aq', 'sn2001el', 'sn2002fk', 'sn2003du',
       'sn2005cf', 'sn2007af', 'sn2007sr', 'sn2009ig', 'sn2011by',
       'sn2011fe', 'sn2012cg', 'sn2012fr'], 
      dtype='|S10')

In [25]:
"""
# 12 SNe with Cepheid distances in Andy's compilation
Out[50]:
array(['sn1998aq', 'sn2001el', 'sn2002fk', 'sn2003du', 'sn2005cf',
       'sn2007af', 'sn2007sr', 'sn2009ig', 'sn2010ai', 'sn2011by',
       'sn2011fe', 'sn2012cg'], 
      dtype='|S10')
"""
0

0

### Function to determine $c z_{\rm cmb}$ from Cepheid $\mu$

These functions are NOT used during any part of the computations in the notebook; I have written here just as part of my library of functions.

In [26]:
def cz_CMB_Cepheid(mu, err_mu, Ho, err_Ho):
    
    # Determine cz_cmb, z_cmb:
    cz_int =  Ho * 10**((mu-25)/5)  # cz_cmb
    zcmb_int = cz_int/c  # z_cmb
    
    # Determine error_cz_cmb, error_z_cmb:
    error_cz_int = cz_int * np.sqrt((err_Ho/Ho)**2 + (np.log(10)*err_mu/5)**2) # error_cz_cmb
    error_zcmb_int = error_cz_int/c  # error_z_cmb
    
    # (z_cmb, error_z_cmb, cz_cmb, error_cz_cmb)
    return zcmb_int, error_zcmb_int, cz_int, error_cz_int

    
# TEST:
print '# sn2003du (data from Riess+16):'
print '#',cz_CMB_Cepheid(32.919, 0.063, 73.24, 1.74)

print ''
print '# sn2005cf (data from Riess+16):'
print '#',cz_CMB_Cepheid(32.263, 0.102, 73.24, 1.74)

# sn2003du (data from Riess+16):
# (0.009369742003029419, 0.00035135265754374572, 2808.977985914033, 105.33287682987176)

# sn2005cf (data from Riess+16):
# (0.006926720004322207, 0.00036461514433908711, 2076.578415973525, 109.3088703454397)

# sn2003du (data from Riess+16):
# (0.009369742003029419, 0.00035135265754374572, 2808.977985914033, 105.33287682987176)

# sn2005cf (data from Riess+16):
# (0.006926720004322207, 0.00036461514433908711, 2076.578415973525, 109.3088703454397)


In [27]:
if NotebookToPlotOnly == False:
    
    HoNew = 72.0; err_HoNew = 1.74

    # err_HoNew = 1.74 comes from assuming the same uncertainty than the 
    # case of  Ho=73.24 +/- 1.74 km/s/Mpc  (Riess+2016).

    print '# sn2003du (data from Riess+16):'
    print '#',cz_CMB_Cepheid(32.919, 0.063, HoNew, err_HoNew)

    print ''
    print '# sn2005cf (data from Riess+16):'
    print '#',cz_CMB_Cepheid(32.263, 0.102, HoNew, err_HoNew)

    print ''
    print '# sn2002fk (data from Riess+16):'
    print '#',cz_CMB_Cepheid(32.523, 0.055, HoNew, err_HoNew)

    print ''
    print '# sn2001el (data from Riess+16):'
    print '#',cz_CMB_Cepheid(31.311, 0.045, HoNew, err_HoNew)

    print ''
    print '# sn2007sr (data from Riess+16):'
    print '#',cz_CMB_Cepheid(31.290, 0.112, HoNew, err_HoNew)

    print ''
    print '# sn2007af (data from Riess+16):'
    print '#',cz_CMB_Cepheid(31.786, 0.046, HoNew, err_HoNew)

    print ''
    print '# sn2009ig (data from Riess+16):'
    print '#',cz_CMB_Cepheid(32.497, 0.081, HoNew, err_HoNew)

    print ''
    print '# sn2011by (data from Riess+16):'
    print '#',cz_CMB_Cepheid(31.587, 0.070, HoNew, err_HoNew)

    print ''
    print '# 2011fe (data from Riess+16):'
    print '#',cz_CMB_Cepheid(29.135, 0.045, HoNew, err_HoNew)


In [28]:
# sn2003du (data from Riess+16):
# (0.009211106283699049, 0.00034780399672242258, 2761.420193689383, 104.269015079639)

# sn2005cf (data from Riess+16):
# (0.006809446208508997, 0.00035970803370978462, 2041.4206164676925, 107.83775558820318)

# sn2002fk (data from Riess+16):
# (0.007675590444195331, 0.00026870678361360224, 2301.08412586663, 80.556267140795939)

# sn2001el (data from Riess+16):
# (0.004392500236022846, 0.00013983623152753925, 1316.838442522869, 41.921847567098084)

# sn2007sr (data from Riess+16):
# (0.00435022573745372, 0.00024778376284622327, 1304.164866686113, 74.283703316158338)

# sn2007af (data from Riess+16):
# (0.005466530725939691, 0.00017567735913141312, 1638.8246830619842, 52.666747308955081)

# sn2009ig (data from Riess+16):
# (0.007584235213199466, 0.00033708985696221771, 2273.6965166152218, 101.05699678557166)

# sn2011by (data from Riess+16):
# (0.004987831728307585, 0.00020095452262929847, 1495.314333919719, 60.244650285254004)

# 2011fe (data from Riess+16):
# (0.0016125448162764855, 5.1335726388381935e-05, 483.428774106686, 15.390063597188481)

## NOTE: I can skip these cells if I want to compute the optimal phase range only.

### Preeliminary writting of  text files

In [29]:
if NotebookToPlotOnly == False:

    # Creation of the datatable text file of (z_CMB, dist_mu, dist_mu_error, delta_mu) 

    InfoSN_NumLinesSkip = 8 # Number info lines in LC files

    # Mean Absolute magnitude determined from histogram of 'appMagTmax_s - mu_s'?:
    # If so it is generated the final 'DistanceMu_Good_AfterCutoffs_Main_.txt' text file, otherwise
    # generate temporal text files.
    if  AbsMagFromHisto == True: textPrefix = ''; underline = ''
    else: textPrefix = 'TMP'; underline = '_' # 'TMP' stands for 'temporal' file.

    textfileMain = open(DirSaveOutput+'DistanceMu_Good_AfterCutoffs_Main_%s%s.txt'%(textPrefix, underline), 'w')
    textfile = open(DirSaveOutput+textPrefix+underline+'DistanceMu_All_BeforeCutoffs_.txt', 'w')
    textfil3 = open(DirSaveOutput+textPrefix+underline+'DistanceMu_Good_AfterCutoffs_z001_.txt', 'w')
    textfil4 = open(DirSaveOutput+textPrefix+underline+'RMS_tmp.txt', 'w')
    textfil5 = open(DirSaveOutput+textPrefix+underline+'DistanceMu_SNe_OutCutoffs_.txt', 'w')
    textfil6 = open(DirSaveOutput+textPrefix+underline+'DistanceMu_Good_AfterCutoffs_Main_NamesOnly_.txt', 'w')

    #------------------------------------------------------------------------------------

    #       Defining the text

    text_001 = '#    %s band  (template method) \n'%BandName
    text01b = '# MAIN DATA TABLE. Used to make further computations and plots \n'

    now = datetime.datetime.now() # Read the time and date right now
    text_timenow = now.strftime("%Y-%m-%d (yyyy-mm-dd)")
    text_Date   = '# On date: %s \n'%text_timenow
    text_Author = '# Data table created by: Arturo Avelino \n'
    text_script = '# Script used: %s \n'%NotebookName
    text_line = '#'+'-'*70 + '\n'

    text02 = '# This datatable is for information only. It is NOT used to compute or plot something else. \n'

    text01 = '# Apparent mag data used to construct the Hubble diagram: %s \n'%KindOfData4HD
    text1 = '# Template used: \n'
    text2 = '# %s \n'%DirTemplate[78:]
    text3 = '# Cosmology used to compute residuals: (Omega_M=%r, Omega_L=%r, w=%r, Ho=%r) \n'%(OmMFix, OmLFix, wFix, HoFix)

    text3_0_0 = '# 0.01 < z_cmb < %r \n'%zcmbUpperLim
    text3_0_1 = '# Cutoffs: z_cmb<%r | %r<dm15<%r | %r<EBVhost<%r | EBV_mw<%r | chi2_dof<%r | Residual<%r \n'%(zcmbUpperLim,
        dm15LowerLim,  dm15UpperLim, EBVhostMin, EBVhostMax, EBVMWLim, chi2_dof_Max, residualMax)
    text3_3 = '# Phase range used of the template: (%r, %r) days \n'%(PhaseMinTemp, PhaseMaxTemp)
    text3_4 = '# Minimal number of data in LC to be considered for the Hubble diagram: %r \n'%MinNumOfDataInLC
    text3_5 = '# Assumed uncertainty in the peculiar velocity: %r km/s \n'%vpecFix
    text4 = '# SN name                        z_CMB         error_zcmb        mu          error_mu       mu_residual    chi2_dof     (1=CfA,2=CSP)   appMag_TBmax  error_appMagTBmax  mu_LCDM  sigma_muLCDM_vPec AbsMagTBmax error_AbsMagTBmax  TBmax       Error_TBmax  PhaseB zhelio        err_zhelio   dm15         error_dm15      EBVhost      error_EBVhost   EBV_MW   error_EBV_MW  Alamb       err_Alamb      R_F          mu_Snoopy     error_muSnoopy   TBmax      Error_TBmax  appMag_TBmax  error_appMagTBmax  Notes   \n' 

    # text4 = '# SN name                        z_CMB     error_zcmb        mu          error_mu      \
    # mu_residual     chi2_dof   (1=CfA,2=CSP)   appMag_TBmax   error_appMagTBmax   mu_LCDM   sigma_muLCDM_vPec  
    # dm15       error_dm15        EBVhost    error_EBVhost      \
    # EBV_MW     error_EBV_MW   mu_Snoopy   error_mu_Snoopy    TBmax     Error_TBmax    Notes \n' 

    #------------------------------------------------------------------ 

    textMethod = '# Method to determine the distance modulus?: %s \n'%Method 
    text_CorrMatrix = '# Use the correlation (exponential kernel) matrix in the template to compute the distance moduli?: (answer in the lines below) \n'
    text_CovMat_MeanMu  = '# CovMat_MeanMu = %s  # correlation matrix to compute the -mean-  distance moduli? \n'%CovMat_MeanMu
    text_CovMat_ErrorMu = '# CovMat_ErrorMu = %s # correlation matrix to compute the -error- distance moduli? \n'%CovMat_ErrorMu
    text_lkernel = '# l_kernel = %r. Lenght scale of the kernel in the covariance matrix for the residual. \n'%(l_kern)
    text_Template = "# Template = %s \n"%TemplateFile
    text_CovMatrix_MeanMu_1  = '# Total CovMat to compute Mean_mu:  CovMatrix_MeanMu = CovMatCorrData_4MeanMu  + Cov_appmag \n'
    text_CovMatrix_MeanMu_2  = '# Total CovMat to compute Mean_mu:  CovMatrix_MeanMu = CovMatCorrData_4MeanMu  + Cov_appmag + Cov_pecvel \n'
    text_CovMatrix_ErrorMu_1 = '# Total CovMat to compute Error_mu: CovMatrix_ErrorMu= CovMatCorrData_4ErrorMu + Cov_appmag + Cov_pecvel \n'
    text_CovMatrix_ErrorMu_2 = '# Total CovMat to compute Error_mu: CovMatrix_ErrorMu= CovMatCorrData_4ErrorMu + Cov_appmag \n'
    text_sigma2mu_1 = '# Variance mu: sigma2_mu = (1/Denominator_ErrorMu) \n'
    text_sigma2mu_2 = '# Variance mu: sigma2_mu = (1/Denominator_ErrorMu) +  sigma2_pec(zcmbInt, err_zcmb, vpecFix) \n'
    # text_IntrinsicDisp_1 = '# Intrinsic dispersion for the case (0 < z < %s) and used to obtain the total photometric \
    # distance modulus uncertainty:\n'%zcmbUpperLim
    # text_IntrinsicDisp_2 = '# Intrinsic dispersion = %s \n'%IntrinsicDisp

    #=====================================================================================

    #       Writting the text

    # 'DistanceMu_Good_AfterCutoffs_Main_.txt'

    textfileMain.write(text_001); textfileMain.write(text01b)

    textfileMain.write(text_line); 
    textfileMain.write(text_Author); textfileMain.write(text_Date); textfileMain.write(text_script);
    textfileMain.write(text_line); 

    textfileMain.write(text01); textfileMain.write(text1); textfileMain.write(text2); textfileMain.write(text3)
    textfileMain.write(text3_0_1)
    textfileMain.write(text3_3); textfileMain.write(text3_4); textfileMain.write(text3_5);
    # if CovMat_MeanMu == True: textfileMain.write(text_lkernel)
    textfileMain.write(text_line); 
    textfileMain.write(textMethod); 
    textfileMain.write(text_CorrMatrix); 
    textfileMain.write(text_CovMat_MeanMu); textfileMain.write(text_CovMat_ErrorMu); textfileMain.write(text_lkernel)
    textfileMain.write(text_Template)
    if Method==1:
        textfileMain.write(text_CovMatrix_MeanMu_1); textfileMain.write(text_CovMatrix_ErrorMu_1); 
        textfileMain.write(text_sigma2mu_1); 
    elif  Method==2:
        textfileMain.write(text_CovMatrix_MeanMu_1); textfileMain.write(text_CovMatrix_ErrorMu_2); 
        textfileMain.write(text_sigma2mu_2); 
    elif  Method==3:
        textfileMain.write(text_CovMatrix_MeanMu_2); textfileMain.write(text_CovMatrix_ErrorMu_1); 
        textfileMain.write(text_sigma2mu_1); 
    elif Method==4:
        textfileMain.write(text_CovMatrix_MeanMu_1); textfileMain.write(text_CovMatrix_ErrorMu_2); 
        textfileMain.write(text_sigma2mu_1);
    elif Method==5 or Method==6:
        textfileMain.write(text_CovMatrix_MeanMu_1); textfileMain.write(text_CovMatrix_ErrorMu_1); 
        textfileMain.write(text_sigma2mu_1); 
    elif Method==7:
        textfileMain.write('#       Distance modulus determination: \n')
        textfileMain.write('# CovMatrix_MeanMu = CovMatCorrData_4MeanMu + Cov_appmag \n')
        textfileMain.write('# appMag_TBmax = Numerator_MeanMu/Denominator_MeanMu \n')
        textfileMain.write('# mu_photo_Analytic = appMag_TBmax - <Average_NIRAbsMag_TBmax> \n')
        textfileMain.write('#       Uncertainty in the distance modulus determination: \n')
        textfileMain.write('# CovMatrix_ErrorMu = CovMatCorrData_4ErrorMu + Cov_appmag \n')
        textfileMain.write('# error_appMag_TBmax = np.sqrt(1/Denominator_ErrorMu)  \n')
        textfileMain.write('# sigma2_mu = (1/Denominator_ErrorMu) \n')
    if Method==6 or Method==7: 
        textfileMain.write('# Using a NORMALIZED template. Distance mu = m_TBmax - M_TBmax. \n')
        textfileMain.write('#       Assumed values for Average_NIRAbsMag_TBmax and error_Average_NIRAbsMag_TBmax: \n')
        textfileMain.write('# <Average_NIRAbsMag_TBmax> = %r. %s \n'%(Average_NIRAbsMag_TBmax, Notes1))
        textfileMain.write('# error_<Average_NIRAbsMag_TBmax> = %r. %s \n'%(error_Average_NIRAbsMag_TBmax, Notes1))
        # textfileMain.write(text_IntrinsicDisp_1); textfileMain.write(text_IntrinsicDisp_2);

    textfileMain.write(text_line); 
    textfileMain.write(text4)

    #----------------------------

    # 'DistanceMu_Good_AfterCutoffs_Main_NamesOnly_.txt'

    textfil6.write('# SN name \n')

    #----------------------------

    # 'DistanceMu_All_BeforeCutoffs_.txt'

    textfile.write(text_001); textfile.write(text02)
    textfile.write(text_line); 
    textfile.write(text_Author); textfile.write(text_Date); textfile.write(text_script);
    textfile.write(text_line); 
    textfile.write(text01); textfile.write(text1); textfile.write(text2); textfile.write(text3)
    textfile.write('# No restrictions on z_cmb, dm15, EBVhost, EBV_mw, chi2_dof, residuals \n')
    textfile.write(text3_3); textfile.write(text3_4); textfile.write(text3_5)
    # if CovMat_MeanMu == True: textfile.write(text_lkernel)
    textfile.write(text_line); 
    textfile.write(textMethod)
    textfile.write(text_CorrMatrix); 
    textfile.write(text_CovMat_MeanMu); textfile.write(text_CovMat_ErrorMu); textfile.write(text_lkernel)
    textfile.write(text_Template)
    if Method==1:
        textfile.write(text_CovMatrix_MeanMu_1); textfile.write(text_CovMatrix_ErrorMu_1); 
        textfile.write(text_sigma2mu_1); 
    elif  Method==2:
        textfile.write(text_CovMatrix_MeanMu_1); textfile.write(text_CovMatrix_ErrorMu_2); 
        textfile.write(text_sigma2mu_2); 
    elif  Method==3:
        textfile.write(text_CovMatrix_MeanMu_2); textfile.write(text_CovMatrix_ErrorMu_1); 
        textfile.write(text_sigma2mu_1); 
    elif Method==4:
        textfile.write(text_CovMatrix_MeanMu_1); textfile.write(text_CovMatrix_ErrorMu_2); 
        textfile.write(text_sigma2mu_1);
    elif Method==5 or Method==6:
        textfile.write(text_CovMatrix_MeanMu_1); textfile.write(text_CovMatrix_ErrorMu_1); 
        textfile.write(text_sigma2mu_1); 
    elif Method==7:
        textfile.write('#       Distance modulus determination: \n')
        textfile.write('# CovMatrix_MeanMu = CovMatCorrData_4MeanMu + Cov_appmag \n')
        textfile.write('# appMag_TBmax = Numerator_MeanMu/Denominator_MeanMu \n')
        textfile.write('# mu_photo_Analytic = appMag_TBmax - <Average_NIRAbsMag_TBmax> \n')
        textfile.write('#       Uncertainty in the distance modulus determination: \n')
        textfile.write('# CovMatrix_ErrorMu = CovMatCorrData_4ErrorMu + Cov_appmag \n')
        textfile.write('# error_appMag_TBmax = np.sqrt(1/Denominator_ErrorMu)  \n')
        textfile.write('# sigma2_mu = (1/Denominator_ErrorMu) \n')
    if Method==6 or Method==7: 
        textfile.write('# Using a NORMALIZED template. Distance mu = m_TBmax - M_TBmax. \n')
        textfile.write('#       Assumed values for Average_NIRAbsMag_TBmax and error_Average_NIRAbsMag_TBmax: \n')
        textfile.write('# <Average_NIRAbsMag_TBmax> = %r. %s \n'%(Average_NIRAbsMag_TBmax, Notes1))
        textfile.write('# error_<Average_NIRAbsMag_TBmax> = %r. %s \n'%(error_Average_NIRAbsMag_TBmax, Notes1))
        # textfile.write(text_IntrinsicDisp_1); textfile.write(text_IntrinsicDisp_2);

    textfile.write(text_line); 
    textfile.write(text4)

    #----------------------------

    textfil3.write(text02); textfil3.write(text01); textfil3.write(text1); textfil3.write(text2) 
    textfil3.write(text3)
    textfil3.write(text3_0_0); textfil3.write(text3_0_1)
    textfil3.write(text3_3); textfil3.write(text3_4); textfil3.write(text3_5)
    # if CovMat_MeanMu == True: textfil3.write(text_lkernel)
    textfil3.write(text_line); 
    textfil3.write(textMethod); 
    textfil3.write(text_CorrMatrix); 
    textfil3.write(text_CovMat_MeanMu); textfil3.write(text_CovMat_ErrorMu); textfil3.write(text_lkernel)
    textfil3.write(text_Template)
    if Method==1:
        textfil3.write(text_CovMatrix_MeanMu_1); textfil3.write(text_CovMatrix_ErrorMu_1); 
        textfil3.write(text_sigma2mu_1); 
    elif  Method==2:
        textfil3.write(text_CovMatrix_MeanMu_1); textfil3.write(text_CovMatrix_ErrorMu_2); 
        textfil3.write(text_sigma2mu_2); 
    elif  Method==3:
        textfil3.write(text_CovMatrix_MeanMu_2); textfil3.write(text_CovMatrix_ErrorMu_1); 
        textfil3.write(text_sigma2mu_1); 
    elif Method==4:
        textfil3.write(text_CovMatrix_MeanMu_1); textfil3.write(text_CovMatrix_ErrorMu_2); 
        textfil3.write(text_sigma2mu_1); 
    textfil3.write(text_line); 
    textfil3.write(text4)

    #----------------------------

    textfil4.write(text_line), textfil4.write('# <- Template range -> \n')
    textfil4.write('# Phase_min  Phase_max   RMS    # SNe   RMS(z>0.01)  # SNe  \n')

    #----------------------------

    textfil5.write('# SNe that are OUTSIDE the following cutoffs-criteria: \n')  
    textfil5.write(text3_0_0); textfil5.write(text3_0_1)
    textfil5.write(text02); textfil5.write(text01); textfil5.write(text1); textfil5.write(text2)
    textfil5.write(text3); 
    textfil5.write(text3_3); textfil5.write(text3_4); textfil5.write(text3_5)
    # if CovMat_MeanMu == True: textfil5.write(text_lkernel)
    textfil5.write(text_line); 
    textfil5.write(textMethod)
    textfil5.write(text_CorrMatrix); 
    textfil5.write(text_CovMat_MeanMu); textfil5.write(text_CovMat_ErrorMu); textfil5.write(text_lkernel)
    textfil5.write(text_Template)
    if Method==1:
        textfil5.write(text_CovMatrix_MeanMu_1); textfil5.write(text_CovMatrix_ErrorMu_1); 
        textfil5.write(text_sigma2mu_1); 
    elif  Method==2:
        textfil5.write(text_CovMatrix_MeanMu_1); textfil5.write(text_CovMatrix_ErrorMu_2); 
        textfil5.write(text_sigma2mu_2); 
    elif  Method==3:
        textfil5.write(text_CovMatrix_MeanMu_2); textfil5.write(text_CovMatrix_ErrorMu_1); 
        textfil5.write(text_sigma2mu_1); 
    elif Method==4:
        textfil5.write(text_CovMatrix_MeanMu_1); textfil5.write(text_CovMatrix_ErrorMu_2); 
        textfil5.write(text_sigma2mu_1); 
    textfil5.write(text_line); 
    textfil5.write(text4)


In [30]:
""" 
textfile.close()
textfileMain.close()
textfil3.close()
textfil4.close()
textfil5.close()
"""
0

0

# Main loop

### Moving the template up or down to match the LC data. The amount of moving up or down corresponds to the photometric distance modulus.

####  In this loop it is used ALL the apparent mag LCs located in "~/1_AllData_InitialFit/AppMag/" through the text file "ListSNe_AppMag_.txt" created and read above.

In [31]:
if NotebookToPlotOnly == False:
    
    # Initializing some quantities
    countGoodSNe = 0; countGoodSNe_z001 = 0; # Counters
    countCfA = 0;     countCSP = 0;     countOthers=0;  countSNe = 0 # Counters
    countCfA_z001= 0; countCSP_z001= 0; countOthers_z001=0 # Counters
    countNoCommented = 0; countCommented = 0;
    countBadSNe = 0; 

    # To be used to compute the RMS:
    mu_resid_array = []; mu_resid_z001_array = []; 
    chi2_list = []; chi2_z001_list = []; 
    SNeName_list = []; SNeName_z001_list = []; 

    # The phase range of the template to be used to determine the distance modulus
    lowerPhase = Template[indexLowerPhase][0]
    upperPhase = Template[indexUpperPhase-1][0]

    ############ THE LOOP ############################

    for j in range(len(list_SNe)): # loop over the SNe
    # for j in range(7):
    # for j in range(10):
    # for j in range(20, 40):

        # Upload the apparent magnitude data for a given SN
        magData = np.genfromtxt(DirAppMag+list_SNe[j][0])
        name = list_SNe[j][0]
        mask = list_SNe[j][1]

        zcmbInt = magData[0,0]
        dm15Int = magData[1,0]
        EBV_MW = magData[3,0]
        EBVhost = magData[4,0]

        err_zcmb = magData[0,1]
        err_dm15 = magData[1,1]
        err_EBVMW = magData[3,1]
        err_EBVhost = magData[4,1]



        # These quantities are not used to compute anything, they are just to be
        # written int the output file
        muSnoopy     = magData[2,0]
        err_muSnoopy = magData[2,1]
        TBmax      = magData[5,0]
        err_TBmax  = magData[5,1]
        zhelio     = magData[0,2]
        err_zhelio = magData[0,3]
        Alamb      = magData[3,2] 
        err_Alamb  = magData[3,3] 
        R_F        = magData[3,4]

        #-------- Peculiar velocity uncertainty -----------------        
        # Create the variable "snName" containing the first 8 (or 7) letters of the SNe file name
        # I use "snName" to compare with the SNe names in 'SNeWithCepheidDistances.txt', so that
        # I will not compute a peculiar velocity uncertainty for those SNe.
        try:
            if   name[7] == '_': snName = name[:7] # To read correctly, e.g., "sn2011B_"
            elif name[7] != '_':
                if is_number(name[7]): snName = name[:15] # To read correctly, e.g., "snf20080514-002"
                else: snName = name[:8]  # To read correctly, e.g., "sn1998bu" 
        except: snName = name[:6]  # To read correctly, e.g., "sn2011B"

        sigma_pecInt = 0 # Reset its value:
        # If this SNe has Cepheid distance, then use vpecFix=0 for it: 
        if snName in ListSNeCepheid['f0']: sigma_pecInt = np.sqrt(sigma2_pec(zcmbInt, err_zcmb, 0))
        else: sigma_pecInt = np.sqrt(sigma2_pec(zcmbInt, err_zcmb, vpecFix))

        #---------------------------
        # If there are at least 3 photometric points in the app mag LC text file, then compute. 
        if len(magData) >= InfoSN_NumLinesSkip + MinNumOfDataInLC:

            #---------------------------
            # Find out if there are at least one datum inside the interpolated phase range of the template
            NumDataInRange = 0
            for i in range(InfoSN_NumLinesSkip, len(magData)): # Loop over all the phases for a given SNe
                if magData[i,0] >= lowerPhase and magData[i,0] <= upperPhase:
                    NumDataInRange = NumDataInRange + 1

            #---------------------------
            # Fit the LCs with at least one data point within the phase range defined for the template.
            if NumDataInRange > 0:

                # Analytical minimization of chi2 to find the best estimate for distance modulus
                # and computing the uncertainty of distance modulus (analytic expression)

                # Resetting some quantities
                Numerator_MeanMu  = 0; Denominator_MeanMu= 0
                Numerator_ErrorMu = 0; Denominator_ErrorMu= 0
                mu_photo_Analytic=0; sigma2_mu=0; stdDev_mu=0
                appMag_TBmax=0; error_appMag_TBmax=0;
                AbsMag_s=0; error_AbsMag_s=0;
                mu_LCDM=0;

                CovMatrix_MeanMu = 0; InvCovMatrix_MeanMu = 0;
                CovMatrix_ErrorMu = 0; InvCovMatrix_ErrorMu =0;
                OffsetToMatchAppMagData = 0; appMag_TBmax = 0;

                #---------------------------    
                # Using the covariance matrix of the photometry to determine the distance modulus:

                # Create arrays for the data that is inside the interpolation 
                # range & within the range (lowerPhase, upperPhase).
                mag_array = [] # apparent magnitude data
                MagTemp_array = [] # template mean value (absolute magnitude)
                error2_appmag = [] # app magnitud errors
                error_temp = [] # population standard deviation of the template at different phases.
                phases = [] # phases inside the interpolation range & within the range (lowerPhase, upperPhase).
                phaseInt = 0
                countGoodPhases = 0; countLowerPhase = 0; 

                for i in range(InfoSN_NumLinesSkip, len(magData)): # Loop over all the photometry for a given SNe
                    if magData[i,0] >= lowerPhase: # Ignore data outside the phase interpolation range:
                        if magData[i,0] <= upperPhase:
                            phaseInt = magData[i,0]
                            mag_array += [magData[i,3]]
                            error2_appmag += [magData[i,4]**2]
                            MagTemp_array += [MTemplInt(phaseInt)]
                            error_temp += [error_MTemplInt(phaseInt)]
                            phases += [phaseInt]
                            countGoodPhases = countGoodPhases + 1
                    else: countLowerPhase = countLowerPhase + 1

                error_temp = np.array(error_temp)


                #-------- Some covariance matrices --------

                # Diagonal variance matrix of the photometric data.
                Cov_appmag = np.diag(error2_appmag)

                # Square covariance matrix of the peculiar velocity uncertainty
                Cov_pecvel = np.ones((countGoodPhases,countGoodPhases))*(sigma_pecInt**2)


                #--------- MEAN photometric distance modulus -------------

                # Covariance matrix of the light-curve photometric data:
                CovMatCorrData_4MeanMu = np.zeros(shape=(countGoodPhases,countGoodPhases)) # Empty array
                for i in range(countGoodPhases):
                    if CovMat_MeanMu == True:
                        for k in range(countGoodPhases):
                            CovMatCorrData_4MeanMu[i,k] = error_temp[i]*error_temp[k]*np.exp( -(((phases[i] - phases[k])/l_kern)**2)/2 ) 
                    elif CovMat_MeanMu == False:
                        CovMatCorrData_4MeanMu[i,i] = error_temp[i]*error_temp[i]

                # Total covariance matrix
                if Method==1 or Method==2 or Method==4 or Method==5 or Method==6 or Method==7: 
                    CovMatrix_MeanMu = CovMatCorrData_4MeanMu + Cov_appmag
                elif Method==3: CovMatrix_MeanMu = CovMatCorrData_4MeanMu + Cov_appmag + Cov_pecvel

                # Inverse of the total covariance matrix
                InvCovMatrix_MeanMu = np.linalg.inv(CovMatrix_MeanMu)

                #---Computing the best estimated photometric distance modulus ---
                # Loop over all the photometry for a given SNe
                # Ignore data outside the interpolation range (lowerPhase, upperPhase):
                for i in range(countGoodPhases): 
                    # Ignore data outside the interpolation range:
                    Numerator_MeanMu = Numerator_MeanMu + (mag_array[i] - MagTemp_array[i])*np.sum(InvCovMatrix_MeanMu[i,:])
                    Denominator_MeanMu = Denominator_MeanMu + np.sum(InvCovMatrix_MeanMu[i,:])

                # Best estimated distance modulus:
                if Method==6 or Method==7: 
                    appMag_TBmax = Numerator_MeanMu/Denominator_MeanMu
                    mu_photo_Analytic = appMag_TBmax - Average_NIRAbsMag_TBmax

                else: 
                    appMag_TBmax = 0
                    mu_photo_Analytic = Numerator_MeanMu/Denominator_MeanMu   

                #--------- UNCERTAINTY in the photometric distance modulus -------------

                # Covariance matrix of the light-curve photometric data to compute ERROR_MU
                CovMatCorrData_4ErrorMu = np.zeros(shape=(countGoodPhases,countGoodPhases)) # Empty array
                for i in range(countGoodPhases):
                    if CovMat_ErrorMu == True:
                        for k in range(countGoodPhases):
                            CovMatCorrData_4ErrorMu[i,k] = error_temp[i]*error_temp[k]*np.exp( -(((phases[i] - phases[k])/l_kern)**2)/2 ) 
                    elif CovMat_ErrorMu == False:
                        CovMatCorrData_4ErrorMu[i,i] = error_temp[i]*error_temp[i]            

                # Total covariance matrix
                if Method==1 or Method==3 or Method==5 : 
                    CovMatrix_ErrorMu = CovMatCorrData_4ErrorMu + Cov_appmag + Cov_pecvel
                elif Method==2 or Method==4 or Method==6 or Method==7: 
                    CovMatrix_ErrorMu = CovMatCorrData_4ErrorMu + Cov_appmag                   

                # Inverse of the total covariance matrix
                InvCovMatrix_ErrorMu = np.linalg.inv(CovMatrix_ErrorMu)            

                #--- Uncertainty of the best estimated distance modulus---
                # Loop over all the photometry for a given SNe
                # Ignore data outside the interpolation range (lowerPhase, upperPhase):
                for i in range(countGoodPhases): 
                    # Ignore data outside the interpolation range:
                    Numerator_ErrorMu = Numerator_ErrorMu + (mag_array[i] - MagTemp_array[i])*np.sum(InvCovMatrix_ErrorMu[i,:])
                    Denominator_ErrorMu = Denominator_ErrorMu + np.sum(InvCovMatrix_ErrorMu[i,:])            

                if Method==7: # uncertainty in the estimated apparent magnitude at T_Bmax
                    error_appMag_TBmax = np.sqrt(1/Denominator_ErrorMu)
                else: error_appMag_TBmax = 0


                # Uncertainty in the estimated distance modulus
                if Method==1 or Method==3 or Method==4 or Method==5 or Method==6: 
                    sigma2_mu = (1/Denominator_ErrorMu)
                elif Method==2: 
                    sigma2_mu = (1/Denominator_ErrorMu) +  (sigma_pecInt**2)

                elif Method==7: 
                    # old1. sigma2_mu = (1/Denominator_ErrorMu) + error_Average_NIRAbsMag_TBmax**2
                    # old2. sigma2_mu = (1/Denominator_ErrorMu) + IntrinsicDisp**2
                    sigma2_mu = 1/Denominator_ErrorMu

                stdDev_mu = np.sqrt(sigma2_mu)               

                #------------------------------------------------------
                # The chi2 function: I define it just to compute the goodness-of-fit to data once 
                # I have computed the best estimated photometric distance modulus.
                # Note that the following definition need the value of "mu_photo_Analytic" computed above.
                chi2Int=0; mu_int=0;

                if Method==6 or Method==7: mu_int = appMag_TBmax
                else: mu_int = mu_photo_Analytic

                for i in range(countGoodPhases):
                    product1=0;
                    for k in range(countGoodPhases): 
                        product1 = product1 + InvCovMatrix_MeanMu[i,k]*(mag_array[k] - MagTemp_array[k] - mu_int)
                    chi2Int = chi2Int + (mag_array[i] - MagTemp_array[i] - mu_int)*product1

                # In case of considering the case where I have just one datum in the LC:
                if countGoodPhases==1: chi2_dof_Int = chi2Int/(countGoodPhases) 
                else: chi2_dof_Int = chi2Int/(countGoodPhases-1)

                #------------------------------------------------------

                # Compute the residual and absolute magnitude value

                mu_LCDM = DistanceMu(magData[0,0], OmMFix, wFix, HoFix)
                mu_resid = mu_photo_Analytic - mu_LCDM

                AbsMag_s = appMag_TBmax - mu_LCDM 
                error_AbsMag_s = np.sqrt(error_appMag_TBmax**2 + sigma_pecInt**2)

                #------------------------------------------------------

                # Defining the subsample label

                if list_SNe[j][0][-9:-6] == 'CfA':
                    indexSample = 1
                elif list_SNe[j][0][-9:-6] == 'CSP':
                    indexSample = 2
                elif list_SNe[j][0][-12:-6] == 'Others':
                    indexSample = 3

                #######################################################

                # WRITTING THE RESULTS TO A TEXT FILE

                # Comment the masked SNe in ListSNe_AppMag_.txt:
                if mask==0: commentText = ''
                else: commentText = '##  '

                # Write the distance modulus for -all- the SNe, no matter any cutoffs.
                # This data will be used to write down the "DistanceMu_All_BeforeCutoffs_.txt" text file

                textfile.write('%s%-30s   \
%.10f  %.10f  %.10f  %.10f  %13.10f  %12.10f        \
%1.f          %.10f  %.10f  \
%.10f  %.10f  \
%.10f  %.10f  %.6f  %.6f      0.00   \
%.10f  %.8f  %.10f  %.10f  \
%13.10f  %.10f  %10.7f  %.7f  %.10f  %.10f  %.10f  \
%.10f  %.10f    \
%.6f  %.6f  %.10f  %.10f # \n'%
                               (commentText, list_SNe[j][0][:-4],
                                zcmbInt, err_zcmb, mu_photo_Analytic, stdDev_mu, mu_resid, chi2_dof_Int, 
                                indexSample, appMag_TBmax, error_appMag_TBmax, 
                                mu_LCDM, sigma_pecInt,
                                AbsMag_s, error_AbsMag_s, TBmax, err_TBmax,
                                zhelio, err_zhelio, dm15Int, err_dm15, 
                                EBVhost, err_EBVhost, EBV_MW, err_EBVMW, Alamb, err_Alamb, R_F,
                                muSnoopy, err_muSnoopy,
                                TBmax, err_TBmax, appMag_TBmax, error_appMag_TBmax ))                        

                countSNe = countSNe + 1 # Counter

                # ------- QUALITY CUTOFFS:  -------
                # This data will be used to write down the "DistanceMu_Good_AfterCutoffs_Main_.txt" text file

                if (zcmbInt < zcmbUpperLim and dm15Int > dm15LowerLim and dm15Int < dm15UpperLim and 
                    EBVhost > EBVhostMin and EBVhost < EBVhostMax and EBV_MW < EBVMWLim and 
                    chi2_dof_Int < chi2_dof_Max and mu_resid < residualMax):
                    textfileMain.write('%s%-30s   \
%.10f  %.10f  %.10f  %.10f  %13.10f  %12.10f        \
%1.f          %.10f  %.10f  \
%.10f  %.10f  \
%.10f  %.10f  %.6f  %.6f      0.00   \
%.10f  %.8f  %.10f  %.10f  \
%13.10f  %.10f  %10.7f  %.7f  %.10f  %.10f  %.10f  \
%.10f  %.10f    \
%.6f  %.6f  %.10f  %.10f # \n'%
                               (commentText, list_SNe[j][0][:-4],
                                zcmbInt, err_zcmb, mu_photo_Analytic, stdDev_mu, mu_resid, chi2_dof_Int, 
                                indexSample, appMag_TBmax, error_appMag_TBmax, 
                                mu_LCDM, sigma_pecInt,
                                AbsMag_s, error_AbsMag_s, TBmax, err_TBmax,
                                zhelio, err_zhelio, dm15Int, err_dm15, 
                                EBVhost, err_EBVhost, EBV_MW, err_EBVMW, Alamb, err_Alamb, R_F,
                                muSnoopy, err_muSnoopy,
                                TBmax, err_TBmax, appMag_TBmax, error_appMag_TBmax ))  

                    textfil6.write('%s \n'%list_SNe[j][0][:-6])

                    mu_resid_array += [mu_resid]
                    chi2_list += [chi2_dof_Int]
                    SNeName_list += [list_SNe[j][0][:-4]]

                    countGoodSNe = countGoodSNe + 1
                    if mask==0: countNoCommented = countNoCommented + 1;
                    else: countCommented = countCommented + 1; 

                    if indexSample == 1: countCfA =  countCfA + 1
                    elif indexSample == 2: countCSP =  countCSP + 1
                    elif indexSample == 3: countOthers =  countOthers + 1

                    #---- Cutoff redshift: z_CMB > 0.01 ---
                    # This data will be used to write down the "DistanceMu_Good_AfterCutoffs_z001_.txt"

                    if zcmbInt > 0.01: 
                        textfil3.write('%s%-30s   \
%.10f  %.10f  %.10f  %.10f  %13.10f  %12.10f        \
%1.f          %.10f  %.10f  \
%.10f  %.10f  \
%.10f  %.10f  %.6f  %.6f      0.00   \
%.10f  %.8f  %.10f  %.10f  \
%13.10f  %.10f  %10.7f  %.7f  %.10f  %.10f  %.10f  \
%.10f  %.10f    \
%.6f  %.6f  %.10f  %.10f # \n'%
                               (commentText, list_SNe[j][0][:-4],
                                zcmbInt, err_zcmb, mu_photo_Analytic, stdDev_mu, mu_resid, chi2_dof_Int, 
                                indexSample, appMag_TBmax, error_appMag_TBmax, 
                                mu_LCDM, sigma_pecInt,
                                AbsMag_s, error_AbsMag_s, TBmax, err_TBmax,
                                zhelio, err_zhelio, dm15Int, err_dm15, 
                                EBVhost, err_EBVhost, EBV_MW, err_EBVMW, Alamb, err_Alamb, R_F,
                                muSnoopy, err_muSnoopy,
                                TBmax, err_TBmax, appMag_TBmax, error_appMag_TBmax ))                                        

                        mu_resid_z001_array += [mu_resid]
                        chi2_z001_list += [chi2_dof_Int]
                        SNeName_z001_list += [list_SNe[j][0][:-4]]
                        countGoodSNe_z001 = countGoodSNe_z001 + 1

                        if indexSample == 1: countCfA_z001 =  countCfA_z001 + 1
                        elif indexSample == 2: countCSP_z001 =  countCSP_z001 + 1
                        elif indexSample == 3: countOthers_z001 =  countOthers_z001 + 1

                else:
                    # This data will be used to write down the "DistanceMu_SNe_OutCutoffs_.txt"
                    textfil5.write('%s%-30s   \
%.10f  %.10f  %.10f  %.10f  %13.10f  %12.10f        \
%1.f          %.10f  %.10f  \
%.10f  %.10f  \
%.10f  %.10f  %.6f  %.6f      0.00   \
%.10f  %.8f  %.10f  %.10f  \
%13.10f  %.10f  %10.7f  %.7f  %.10f  %.10f  %.10f  \
%.10f  %.10f    \
%.6f  %.6f  %.10f  %.10f # \n'%
                               (commentText, list_SNe[j][0][:-4],
                                zcmbInt, err_zcmb, mu_photo_Analytic, stdDev_mu, mu_resid, chi2_dof_Int, 
                                indexSample, appMag_TBmax, error_appMag_TBmax, 
                                mu_LCDM, sigma_pecInt,
                                AbsMag_s, error_AbsMag_s, TBmax, err_TBmax,
                                zhelio, err_zhelio, dm15Int, err_dm15, 
                                EBVhost, err_EBVhost, EBV_MW, err_EBVMW, Alamb, err_Alamb, R_F,
                                muSnoopy, err_muSnoopy,
                                TBmax, err_TBmax, appMag_TBmax, error_appMag_TBmax ))                               

                    countBadSNe = countBadSNe + 1


                #######################################################
                #
                # Plotting the LC data and the fit with the template for a given SN

                # Offset to displace the template to match apparent magnitude data
                if Method==7: OffsetToMatchAppMagData = appMag_TBmax
                else: OffsetToMatchAppMagData = mu_photo_Analytic


                # Some definitions to plot the template in the range phase
                x = Template[indexLowerPhase:indexUpperPhase,0] 
                y_pred = Template[indexLowerPhase:indexUpperPhase,1] + OffsetToMatchAppMagData

                if TempType == 'MWA':
                    sigma = np.sqrt(Template[indexLowerPhase:indexUpperPhase,3])
                else: 
                    sigma = Template[indexLowerPhase:indexUpperPhase,2]

                # Creating the plot 
                plt.figure()

                # Standard deviation
                plt.fill(np.concatenate([x, x[::-1]]),
                        np.concatenate([y_pred - 1*sigma,
                                      (y_pred + 1*sigma)[::-1]]),
                        # np.concatenate([y_pred - 1 * sigma,
                        #                (y_pred + 1 * sigma)[::-1]]),
                        alpha=0.3, fc='g', ec='None')

                # PLOT GP TEMPLATE mean
                plt.plot(x, y_pred, 'k-', lw=2, alpha=0.4)

                #-------------------

                # Plot the photometric LC data (app mag) for the given SNe 

                plt.errorbar(magData[InfoSN_NumLinesSkip:,0], magData[InfoSN_NumLinesSkip:,3], 
                             magData[InfoSN_NumLinesSkip:,4], ls='', fmt='r.', alpha=1,  markersize=8)

                #-------------------

                plt.grid(True)

                #-- Defining the y-limits for plotting
                y_lowerlim = MinMagTempPlot + OffsetToMatchAppMagData
                y_upperlim = Template[indexMaxTempl,1]+ OffsetToMatchAppMagData

                plt.ylim(y_lowerlim+0.7, y_upperlim-0.5)
                plt.xlim(x_RangePlots)

                if Method==6 or Method==7:
                    plt.text(-7, y_lowerlim-0.7, 'Best fit: $m_{T_{Bmax}}$ = %r $\pm$ %r'%(round(appMag_TBmax,3), 
                                                                              round(np.sqrt(error_appMag_TBmax),3))) 

                plt.text(-7, y_lowerlim-0.5, '$\mu$ = %r $\pm$ %r'%(round(mu_photo_Analytic,3), 
                                                                              round(np.sqrt(sigma2_mu),3)))  
                plt.text(-7, y_lowerlim-0.3, '$z_{CMB}$ = %r'%round(magData[0,0],5))
                plt.text(-7, y_lowerlim-0.1, '$\Delta \mu$ = %r'%round(mu_resid,3))
                if Chi2dofPrint == True:
                    plt.text(-7, y_lowerlim+0.1, '$\chi^2_{dof}$ = %r'%round(chi2_dof_Int,2))

                plt.title(list_SNe[j][0][:-4])

                plt.xlabel(r'phase = (MJD-$T_{Bmax}$)/(1+$z_{\rm hel}$)')
                plt.ylabel('apparent magnitude')

                if (zcmbInt < zcmbUpperLim and dm15Int > dm15LowerLim and dm15Int < dm15UpperLim and 
                    EBVhost > EBVhostMin and EBVhost < EBVhostMax and EBV_MW < EBVMWLim and
                    chi2_dof_Int < chi2_dof_Max and mu_resid < residualMax):
                    if zcmbInt > 0.01:
                        plt.savefig(DirSaveOutput+'Plot_Fits/AfterCutoffs_z001/'+'%s_FitMu.png'%(list_SNe[j][0][:-4]))
                    else:
                        plt.savefig(DirSaveOutput+'Plot_Fits/AfterCutoffs_z0/'+'%s_FitMu.png'%(list_SNe[j][0][:-4]))
                else:
                    plt.savefig(DirSaveOutput+'Plot_Fits/OutCutoffs/'+'%s_FitMu.png'%(list_SNe[j][0][:-4]))

                plt.close()

                # <--- End plotting data ----

                print 'Done:  %s'%list_SNe[j][0]
                # print ''

    # <--- End loop

    #######################################################

    #     RMS computation

    rms = np.sqrt(np.mean(np.square(mu_resid_array)))
    rms_z001 = np.sqrt(np.mean(np.square(mu_resid_z001_array)))
    # rms_z001_chi2_Res = np.sqrt(np.mean(np.square(mu_resid_z001_chi2_Res_array)))

    # Write to the RMS text file
    textfil4.write('%6.2f       %5.2f      %.4f  %3.f     %.4f      %3.f  \n'%(lowerPhase, upperPhase, 
                                                      rms, countGoodSNe, rms_z001, countGoodSNe_z001))

    #=======================================================

    # text7 = '# Total number of SNe in this list: %r (CfA=%r, CSP=%r, Others=%r) \n'%

    text7 = '# Total number of SNe in this list: %r (CfA=%r, CSP=%r, Others=%r) \n'%(
        countGoodSNe, countCfA, countCSP, countOthers)
    text8 = '# Total number of SNe with z >0.01: %r (CfA=%r, CSP=%r, Others=%r) \n'%(
        countGoodSNe_z001, countCfA_z001, countCSP_z001, countOthers_z001)
    text_08_01 = "# %s SNe no commented automatically (##). \n"%countNoCommented 
    text_08_02 = "# %s SNe were commented automatically (##). \n"%countCommented 
    text_22 = "# %s SNe were commented automatically (##). \n"%countCommented 

    text9  = '# RMS: %r (all the SNe in this file) \n'%round(rms,4)
    text10 = '# RMS: %r (SNe z > 0.01) \n'%round(rms_z001,4)

    text11 = '# Largest chi2: %r, SNe: %s \n'%(
        round(max(chi2_list),4), SNeName_list[chi2_list.index(max(chi2_list))]   )
    text12 = '# Largest chi2 (z>0.01): %r, SNe: %s \n'%(
        round(max(chi2_z001_list),4),
        SNeName_z001_list[chi2_z001_list.index(max(chi2_z001_list))] )
    # text12_1 = '# Largest chi2: %r, SNe: %s \n'%(round(max(chi2_z001_chi2_Res_list),4), 
    #   SNeName_z001_chi2_Res_list[chi2_z001_chi2_Res_list.index(max(chi2_z001_chi2_Res_list))] )

    text13 = '# Largest upper residual: %r, SNe: %s \n'%(
        round(max(mu_resid_array),4), SNeName_list[mu_resid_array.index(max(mu_resid_array))]   )
    text14 = '# Largest lower residual: %r, SNe: %s \n'%(
        round(min(mu_resid_array),4),SNeName_list[mu_resid_array.index(min(mu_resid_array))]  )

    text15 = '# Largest upper residual (z>0.01): %r, SNe: %s  \n'%(
        round(max(mu_resid_z001_array),4),
        SNeName_z001_list[mu_resid_z001_array.index(max(mu_resid_z001_array))]   )
    # text15_1 = '# Largest upper residual: %r, SNe: %s  \n'%(
    #    round(max(mu_resid_z001_chi2_Res_array),4), 
    #   SNeName_z001_chi2_Res_list[mu_resid_z001_chi2_Res_array.index(max(mu_resid_z001_chi2_Res_array))]   )

    text16 = '# Largest lower residual (z>0.01): %r, SNe: %s  \n'%(
        round(min(mu_resid_z001_array),4), 
        SNeName_z001_list[mu_resid_z001_array.index(min(mu_resid_z001_array))]   )
    # text16_1 = '# Largest lower residual: %r, SNe: %s  \n'%(
    #  round(min(mu_resid_z001_chi2_Res_array),4), 
    #  SNeName_z001_chi2_Res_list[mu_resid_z001_chi2_Res_array.index(min(mu_resid_z001_chi2_Res_array))]   )


    textfile.write('# Total number of SNe in this file: %r \n'%countSNe)

    #--------------------
    textfileMain.write(text_line);
    textfileMain.write(text7); textfileMain.write(text8); 
    textfileMain.write(text_08_01); textfileMain.write(text_08_02); 
    textfileMain.write(text9); textfileMain.write(text10)
    textfileMain.write(text11); textfileMain.write(text12); textfileMain.write(text13); 
    textfileMain.write(text14); textfileMain.write(text15); textfileMain.write(text16);
    #--------------------

    textfil3.write(text8); textfil3.write(text10); textfil3.write(text12); textfil3.write(text15)
    textfil3.write(text16)

    # textfil3_1.write(text8_1); textfil3_1.write(text10_1); textfil3_1.write(text12_1); textfil3_1.write(text15)
    # textfil3_1.write(text16)

    textfil5.write('# Total number of SNe in this file: %r \n'%countBadSNe)

    textfile.close()
    textfileMain.close()
    textfil3.close()
    textfil4.close()
    textfil5.close()
    textfil6.close()

    print ' '
    print '--- All done smoothly, DistanceModuli_*.txt created ---'
    print 'Number of SNe before cutoffs = ', countSNe
    print 'Number of SNe after cutoffs = ', countGoodSNe
    print 'Number of SNe after cutoffs AND 0.01 < z < 0.06 = ', countGoodSNe_z001
    print text7, text_08_01, text_22


# Reading the datatable I've just created above

#### This will be used to determine the cut based on $\chi^2$ and to determine the mean absolute value

In [32]:
import os # To use command line like instructions
import glob # To read the files in my directory

#-- Array of minimum redshift to consider --
zCMB_Min = np.array([0, 0.01])
zCMB_Min_Label = ['z0', 'z001']

#-------------------------------------------------------

# Change the working directory where the data files are located
os.chdir(DirSaveOutput)

# Reset the main array container to avoid using the one from 
# the previous run.
DistMu_array = np.array([0])
print "# 'DistMu_array' resetted."

# Reading the data files in that folder 
DistanceMuFiles = glob.glob('DistanceMu_Good_After*.txt')

# Check if 'DistanceModuli_Notes_.txt' is already there, otherwise read
# the 'DistanceModuli_.txt' file.
if 'DistanceMu_Good_AfterCutoffs_Main_Notes_.txt' in DistanceMuFiles:
    DistMu_array = np.genfromtxt(DirSaveOutput+
                        'DistanceMu_Good_AfterCutoffs_Main_Notes_.txt', 
                                 dtype=['S30',
               float,float,float,float,float,float,float,float,float,float,
               float,float,float,float,float,float,float,float,float,float,
               float,float,float,float,float,float,float,float,float,float,
               float,float,float])
    print 'Reading the file:  < DistanceMu_Good_AfterCutoffs_Main_Notes_.txt >'
    
elif 'DistanceMu_Good_AfterCutoffs_Main_.txt' in DistanceMuFiles:
    DistMu_array = np.genfromtxt(DirSaveOutput+
                        'DistanceMu_Good_AfterCutoffs_Main_.txt',
                                 dtype=['S30',
               float,float,float,float,float,float,float,float,float,float,
               float,float,float,float,float,float,float,float,float,float,
               float,float,float,float,float,float,float,float,float,float,
               float,float,float])
    print '# Reading the file: < DistanceMu_Good_AfterCutoffs_Main_.txt >'
    
elif 'DistanceMu_Good_AfterCutoffs_Main_TMP_Notes_.txt' in DistanceMuFiles:
    DistMu_array = np.genfromtxt(DirSaveOutput+
                        'DistanceMu_Good_AfterCutoffs_Main_TMP_Notes_.txt',
                                 dtype=['S30',
               float,float,float,float,float,float,float,float,float,float,
               float,float,float,float,float,float,float,float,float,float,
               float,float,float,float,float,float,float,float,float,float,
               float,float,float])
    print '# Reading the file: < DistanceMu_Good_AfterCutoffs_Main_TMP_Notes_.txt >'
    
elif 'DistanceMu_Good_AfterCutoffs_Main_TMP_.txt' in DistanceMuFiles:
    DistMu_array = np.genfromtxt(DirSaveOutput+
                        'DistanceMu_Good_AfterCutoffs_Main_TMP_.txt',
                                 dtype=['S30',
               float,float,float,float,float,float,float,float,float,float,
               float,float,float,float,float,float,float,float,float,float,
               float,float,float,float,float,float,float,float,float,float,
               float,float,float])
    print '# Reading the file: < DistanceMu_Good_AfterCutoffs_Main_TMP_.txt >'

else: print '# < DistanceMu_Good_AfterCutoffs_Main_.txt > file not found!!!'

print '# Number of SNe in the file = ', len(DistMu_array)

# 'DistMu_array' resetted.
# Reading the file: < DistanceMu_Good_AfterCutoffs_Main_.txt >
# Number of SNe in the file =  29


In [33]:
# Reading the file: < DistanceMu_Good_AfterCutoffs_Main_.txt >
# Number of SNe in the file =  63

------

# $\chi^2_{d.o.f.}$ histogram

In [34]:
# Create 2 lists: for z<0 and z<0.01

if NotebookToPlotOnly == False:
    
    chi2dof_z0_list = []
    chi2dof_z001_list = []

    for i in range(len(DistMu_array)):

        name = DistMu_array[i][0]
        zcmbInt = DistMu_array[i][1]
        chi2dof_z0_list += [DistMu_array[i][6]]

        if zcmbInt > 0.01:
            chi2dof_z001_list += [DistMu_array[i][6]]


    print 'len(chi2dof_z0_list) = ', len(chi2dof_z0_list)
    print 'len(chi2dof_z001_list) = ', len(chi2dof_z001_list)
    # len(chi2dof_z001_list) =  78

In [35]:
# Plotting the histogram of chi2_dof

if NotebookToPlotOnly == False:
    
    # countChi2dofPlot = countChi2dofPlot + 1
    fontsizeText = 15

    BinSize = 45

    for zz in zCMB_Min:
        fig = plt.figure()

        if zz==0.0: plt.hist(chi2dof_z0_list, BinSize, histtype=u'barstacked')
        elif zz==0.01: plt.hist(chi2dof_z001_list, BinSize, histtype=u'barstacked')        

        plt.xlabel(r'$\chi^2_{\rm{d.o.f}}$', fontsize=fontsizeText+2)
        plt.ylabel('Frequency', fontsize=fontsizeText)
        plt.title(r'Histogram $\chi^2_{\rm{d.o.f}}$ (%r<z<%r)'%(zz, zcmbUpperLim), fontsize=fontsizeText)
        plt.text(15, 10, 'Bin size = %r'%BinSize)

        plt.grid(True)

        plt.savefig('Plot_histo_chi2dof_BinSize%r_z%r_.png'%(BinSize, zz))
        plt.close()

In [36]:
if NotebookToPlotOnly == False: plt.close(); plt.close();

In [37]:
# Plotting the values of chi2dof as lines

if NotebookToPlotOnly == False:
    
    # Array of zeros just for the y-axis values.
    zero_z0_np = np.zeros(len(chi2dof_z0_list))
    zero_z001_np = np.zeros(len(chi2dof_z001_list))

    for zz in zCMB_Min:
        fig = plt.figure(figsize=(12,4))

        if  zz==0.0:
            plt.plot(chi2dof_z0_list, zero_z0_np, ls='None', marker='|', ms=30, markeredgewidth=2, color='r',  alpha=1)
        elif zz==0.01:
            # plt.plot(chi2dof_z001_list, zero_np, ls='None', marker='o', ms=12, color='r',  alpha=0.3)
            plt.plot(chi2dof_z001_list, zero_z001_np, ls='None', marker='|', ms=30, markeredgewidth=2, color='r',  alpha=1)

        plt.xlabel(r'$\chi^2_{\rm{d.o.f}}$', fontsize=fontsizeText+2)
        plt.ylabel('No meaning', fontsize=fontsizeText)
        plt.title(r'$\chi^2_{\rm{d.o.f}}$ (%r<z<%r)'%(zz,zcmbUpperLim), fontsize=fontsizeText)

        fig.tight_layout()

        plt.ylim(-1, 1)

        plt.grid(True)

        plt.savefig('Plot_histo_chi2dof_Points_z%r_.png'%(zz))
        plt.close()

In [38]:
if NotebookToPlotOnly == False: plt.close(); plt.close();

# Determining average Absolute magnitude  of the sample

### Plotting the histogram with the Gaussian estimation

In [39]:
# Plotting the histogram with the Gaussian estimation

if NotebookToPlotOnly == False:
    
    from scipy.stats import norm
    import matplotlib.mlab as mlab
    
    # best fit of data.
    # 'norm.fit' simply compute the mean and standard devation of the sample.
    (Average_NIRAbsMag_TBmax, error_Average_NIRAbsMag_TBmax) = norm.fit(DistMu_array['f12'])

    #------ Plot -----------

    # the histogram of the data
    n, bins, patches = plt.hist(DistMu_array['f12'], 20, normed=True, facecolor='green', alpha=0.5)
    # n, bins, patches = plt.hist(DistMu_array['f12'], 20, normed=False, facecolor='green', alpha=0.5)


    # add a 'best fit' Gaussian line
    y = mlab.normpdf(bins, Average_NIRAbsMag_TBmax, error_Average_NIRAbsMag_TBmax )
    l = plt.plot(bins, y, 'r--', linewidth=2)

    plt.xlabel(r'$\hat{m}_{\rm Bmax} - \mu_{\Lambda{\rm CDM}}$')
    plt.ylabel('Probability')
    plt.title(r'Histogram. (mean=%.2f, std dev = %.2f)' %(Average_NIRAbsMag_TBmax, error_Average_NIRAbsMag_TBmax))

    plt.grid(True)
    plt.savefig(DirSaveOutput+'Plot_histo_AbsMag_.png')
    plt.close()

In [40]:
if NotebookToPlotOnly == False: plt.close(); plt.close();

In [41]:
if NotebookToPlotOnly == False:
    
    # The results that I will use now.
    # NOTE: This value is independent of what I have assume about "Average_NIRAbsMag_TBmax" and
    # "error_Average_NIRAbsMag_TBmax" at the top of the notebook in the User section.
    # NOTE: The values of "Average_NIRAbsMag_TBmax" and "error_Average_NIRAbsMag_TBmax" do NOT 
    # depend on the value of "vpecFix".

    # ------------------------------
    now = datetime.datetime.now() # Read the time and date right now
    text_timenow = now.strftime("%Y-%m-%d; %H:%M hrs.")
    text_Date   = '# On date: %s \n'%text_timenow
    # ------------------------------

    print '#', '-'*30
    print '#  ', BandName, 'band | vpecFix =', vpecFix, 'km/s | 0 < z <', zcmbUpperLim
    # print '# (mean abs mag, std deviation)\n' 
    # print ""

    print "# Average_NIRAbsMag_TBmax = %s;  # %s"%(Average_NIRAbsMag_TBmax, text_timenow)
    print "# error_Average_NIRAbsMag_TBmax = %s;"%(error_Average_NIRAbsMag_TBmax)

#### Weighted averages

In [42]:
# Inverse-variance weights array

if NotebookToPlotOnly == False:

    WeightsInvVar = 1/np.square(DistMu_array['f13'])

    # Inverse-variance weighted average
    WeightedAbsMag = np.average(DistMu_array['f12'], weights=WeightsInvVar )

    # Useful definitions to determine the weighted population standard deviation
    V1 = np.sum(WeightsInvVar)
    V2 = np.sum(np.square(WeightsInvVar))

    product2 = 0
    # Computing the unbiased weighted population standard deviation
    for i in range(len(DistMu_array['f12'])):
        # print i
        absMagInt = DistMu_array['f12'][i]
        product2 = product2 + WeightsInvVar[i]*(absMagInt-WeightedAbsMag)**2

    error_WeightedAbsMag = np.sqrt(product2/(V1 - (V2/V1)))


In [43]:
# Compute some other values

# print '#'+'-'*30
# print '#   ', BandName, 'band   Pec velocity =', vpecFix, 'km/s'
# print '#', np.mean(DistMu_array['f12']), '= mean abs mag'
# print '#', np.median(DistMu_array['f12']), '= median' 
# print '#', WeightedAbsMag, '= Weighted Abs Mag'
# print '#', np.sqrt(1/np.sum(WeightsInvVar) ), '= uncertainty in the weighted average'
# print '#', np.std(DistMu_array['f12']), '= population standard deviation'
# print '#', error_WeightedAbsMag, '= unbiased weighted population standard deviation'

-------

# HUBBLE DIAGRAM PLOTS

- Note that in all the rest of the the following cells, it is NOT computed or used at all the values of (Average_NIRAbsMag_TBmax, error_Average_NIRAbsMag_TBmax).


-  (SEE UPDATED INFORMATION BELOW) If I need to remove an outlier in any of the "any YJHK", YJHK, YJH or JHK Gaussian-Process or template Hubble diagrams, then: 


1. first I need to "comment" that SN in 'DistanceMu_Good_AfterCutoffs_Main_.txt' in all the bands, 
2. then, rerun the "13_TotalDistanceMu_vXX.ipynb" iPython notebook to recompute the covariance matrices and their inverse matrix from the remaining SNe.
3. plot the Hubble diagram using "11_DistanceMu_HubbleDiagram_vXX.ipynb" notebook (this section).

UPDATED INFORMATION ABOUT THE CASE OF REMOVING OUTLIERS IN THE "any YJHK", YJHK, YJH or JHK Gaussian-Process or template Hubble diagrams:

The process described above is the correct to remove outliers, however, for simplicity when making different experiments removing a lot of SNe (specially when comparing the NIR vs SALT2 Hubble diagrams from SNe that pass SALT2 cutoffs) I'm recalibrating the Hubble diagram residual by simply determining the value that allows to minimize the weighted *average* in the Hubble residual plot (exactly as done for the SALT2 Hubble diagram). This is done by using the simple single instruction:

elif BandMax == 'NIRmax' and PlotTotalMu == True:

     delta_Mo = SimplexResult_1[0]

If I want to use the correct procedure to remove outliers in the "GP NIR any band HD", I simply have to follow the instructions indicated and comment the instruction above.
The drawback of this method is that the uncertainties are less consistent with scatter in the Hubble diagram, i.e., chi^2_dof is not close to one.


'DistanceModuli_Notes_.txt': The advantage of having a text file listing all the SNe is that I can easily "comment" the outliers SNe in the Hubble diagram and residual.

### USER 

In [44]:
# Setting just to label the plots accordingly.

# Band to use as the reference peak maximum? Options: (NIRmax, Bmax).
# For the template method to determine the distance modulus I use BandMax = 'Bmax'
# For the GP method to determine the distance modulus I use BandMax = 'NIRmax'
# "Bmax_GP": Determine the distance moduli at B-band max but using the GP method.

# WATCH OUT!: Everytime I change between (NIRmax, Bmax, Snoopy, SALT2) Hubble diagrams,
# I must reset this notebook because in each case it is used different values for 
# (Ho, Omega_matter, w).

BandMax = 'NIRmax'  # (NIRmax, Bmax_GP, Bmax, Snoopy, SALT2).

#   LABELS IN THE PLOT'S TITLE
#      LOW-Z
if BandMax == 'SALT2': BandName = 'Optical'
if BandMax == 'Snoopy': BandName = 'BVR'  # ORIGINAL
# if BandMax == 'Snoopy': BandName = 'Optical+NIR'  # TEMPORAL
# if BandMax == 'Snoopy': BandName = 'NIR'  # TEMPORAL
# if BandMax == 'Snoopy': BandName = 'Y'  # TEMPORAL


#    RAISINs
# if BandMax == 'Snoopy': BandName = 'Optical' # RAISINs
# if BandMax == 'Snoopy': BandName = 'Opt+NIR' # RAISINs

#------- "Total" distance modulus ---------

# Plot the "total" distance modulus derived from the three distance modulus computed 
# from each band?
# To put TRUE, first I need to have already computed the distance moduli for each SNe 
# in the YJHK bands!

PlotTotalMu = False    # True, False

# "PlotTotalMu = True"  is only valid when BandMax = NIRmax, Bmax, Bmax_GP.
# "PlotTotalMu = False" if I'm plotting the SNoopy or SALT2 distance moduli.
if BandMax == 'Snoopy' or BandMax == 'SALT2':
    PlotTotalMu = False

# - "AllBands": If there is not distance mu estimation for a given SN, then 
# use the 3, 2 or 1 band that it was observed
# - "YJH": Consider the SNe that have distance mu estimation in these 
# 3 bands ONLY, and so on.

if PlotTotalMu == True: BandsCombination = 'AllBands'   # (AllBands, JH, YJH, JHK,  YJHK)
else: BandsCombination = ''
    
#####################################################

#    Plot RAISINs?

# Set the limits and labels in the plot if I'm plotting RAISINs.
# False = plot the low-z sample.
plot_raisins = False

if plot_raisins == True:
    zcmbUpperLim = 0.65  # (0.6, 0.65, anything else)
    xlimPlots = 0.2, zcmbUpperLim+0.003 # 
    ylimPlots = 40.0, 44.5
    
#####################################################

#    PLOT OPTIONS
# Resolution in the Hubble diagram plots
ResolutionPlot_HD = 150  # dpi

# Settings error bars:
MyPointSize = 6  
MyLineWidth = 1  
MyCapeSize = 2

## Automatic

In [45]:
# Apparent magnitude, determined from GP interpolation, template, snoopy.?
    
if BandMax == 'Bmax':
    AppMag_method = 'Template to compute the app mags'  
    HoFix = 73.24 # Valued used by default in the low-z paper
    OmMFix = 0.28 # Omega_Matter
    OmLFix = 0.72 # Omega_Lambda
    wFix = -1 # Dark energy EoS
    
elif BandMax == 'Bmax_GP':
    AppMag_method = 'Gaussian Process to compute the app mags'  
    HoFix = 73.24 # Valued used by default in the low-z paper
    OmMFix = 0.28 # Omega_Matter
    OmLFix = 0.72 # Omega_Lambda
    wFix = -1 # Dark energy EoS
    
elif BandMax == 'NIRmax':
    AppMag_method = 'Gaussian Process to compute app mags'
    HoFix = 73.24 # Valued used by default in the low-z paper
    OmMFix = 0.28 # Omega_Matter
    OmLFix = 0.72 # Omega_Lambda
    wFix = -1 # Dark energy EoS
elif BandMax == 'Snoopy':
    AppMag_method = 'SNooPy fit'
    # Redefing some variables: 
    HoFix = 72 # km/s. Ho=72 is Snoopy's default value
    OmMFix = 0.28 # Omega_Matter
    OmLFix = 0.72 # Omega_Lambda
    wFix = -1 # Dark energy EoS
elif BandMax == 'SALT2':
    AppMag_method = 'SALT2 fit'
    HoFix = 73.24 # Valued used by default in the low-z paper
    OmMFix = 0.28 # Omega_Matter
    OmLFix = 0.72 # Omega_Lambda
    wFix = -1 # Dark energy EoS

#----------------------------------------------

print "# Hubble diagram type: %s"%BandMax
print "# PlotTotalMu? (when combined YJHK bands only) = %s \n"%PlotTotalMu
print "# Ho = %s km/s/Mpc."%HoFix
print "# Omega_Matter = %s"%OmMFix
print "# Omega_Lambda = %s"%OmLFix

# Hubble diagram type: NIRmax
# PlotTotalMu? (when combined YJHK bands only) = False 

# Ho = 73.24 km/s/Mpc.
# Omega_Matter = 0.28
# Omega_Lambda = 0.72


In [46]:
# Reading the datafile with the TOTAL distance modulus inferred from
# the distance moduli coming from each band.

if PlotTotalMu and BandMax == 'NIRmax': Method_folder = 'GaussianProcess'
if PlotTotalMu and BandMax == 'Bmax': Method_folder = 'Template'
if PlotTotalMu and BandMax == 'Bmax_GP': Method_folder = 'GP_Bmax'

if PlotTotalMu: 
    # Change the working directory where the datafiles are located
    DirSaveOutput = DirMain_1+'AllBands/Plots/HubbleDiagram/%s/%s/'%(
        Method_folder, BandsCombination)
    if not os.path.exists(DirSaveOutput): os.makedirs(DirSaveOutput)
    
    DirDataTotalMu = DirMain_1+'AllBands/Plots/HubbleDiagram/'+Method_folder+'/'
    os.chdir(DirDataTotalMu)

    try:
        DistMu_array = np.genfromtxt('Table_TotalMu_%s_Notes_.txt'%BandsCombination,
                                    dtype=['S30', float,float,float,float,float,float,float])
    except: 
        DistMu_array = np.genfromtxt('Table_TotalMu_%s_.txt'%BandsCombination,
                                    dtype=['S30', float,float,float,float,float,float,float])
        
    BandName = 'All'
    
    print '# Number of SNe in PlotTotalMu file: %s.'%len(DistMu_array)
# Number of SNe in PlotTotalMu file: 30.

#### Determine the value needed to have a weighted average of zero in the Hubble residual 

Used for SALT2 Hubble diagram


In [47]:
# Define the average in the absolute value of the residual
# These functions are going to be minimized.

# AbsResidual values
mu_1_np = DistMu_array['f3']
z_1_np  = DistMu_array['f1']
res_mu_np_int = mu_1_np - DistanceMuVector(z_1_np, OmMFix, wFix, HoFix)

err_mu_1_np = DistMu_array['f4']

# Inverse-variance weighted average function to be minimized 
def WeightedAbsResidual_fun(deltaMo, UpLimit, LowLimit, AbsResidual_Roof):
    residuals_int = res_mu_np_int +  deltaMo
    WeightedAbsResidual_int = np.average(residuals_int, weights=err_mu_1_np )
    
    if deltaMo < UpLimit and deltaMo > LowLimit:  
        # For some unknown reason, simplex is search the maximum instead of
        # the minimimum, so I had to define the average absolute mag as 1/WeightedAbsResidual
        # WeightedAbsResidual_final = 1/WeightedAbsResidual_int
        WeightedAbsResidual_final = WeightedAbsResidual_int
        
    else: WeightedAbsResidual_final = AbsResidual_Roof
        
    return abs(WeightedAbsResidual_final)

#-----------------------------------------------------

# Simple average function to be minimized 
def AverageAbsResidual_fun(deltaMo, UpLimit, LowLimit, AbsResidual_Roof):
    residuals_int = res_mu_np_int + deltaMo
    # AverageAbsResidual_int = np.mean(residuals_int)
    AverageAbsResidual_int = np.sum(residuals_int)/len(residuals_int)
     
    if deltaMo < UpLimit and deltaMo > LowLimit: 
        # For some unknown reason, simplex is search the maximum instead of
        # the minimimum, so I had to define the average absolute mag as 1/AverageAbsResidual_int
        # AverageAbsResidual_final = 1/AverageAbsResidual_int
        AverageAbsResidual_final = AverageAbsResidual_int
        
    else: AverageAbsResidual_final = AbsResidual_Roof

    return abs(AverageAbsResidual_int)

In [48]:
# Determine the value of the constant deltaMo in order to obtain a
# weighted average value of zero in the Hubble residual

import scipy.optimize

# Search limits:
# UpLimit = 0.2; LowLimit = -0.2
UpLimit = 3; LowLimit = -3

# Assume this value for deltaMo in case of the search is outside the range
# indicated above.
Residual_Roof = 10 

WeightedAbsResidual_Out = scipy.optimize.minimize_scalar(WeightedAbsResidual_fun, 
                                                    args=(UpLimit, LowLimit, Residual_Roof))

AverageAbsResidual_Out = scipy.optimize.minimize_scalar(AverageAbsResidual_fun, 
                                                    args=(UpLimit, LowLimit, Residual_Roof))

# Redefining the values:
SimplexResult_1 = [WeightedAbsResidual_Out['x'] ]
SimplexResult_2 = [AverageAbsResidual_Out['x'] ]

print WeightedAbsResidual_Out
print '#', SimplexResult_1, ' = value of delta_Mo that minimize the Hubble residual.'
print 

# print AverageAbsResidual_Out
# print '#', SimplexResult_2, ' = value of delta_Mo that minimize the Hubble residual.'

     fun: 1.1444956596785391e-10
    nfev: 30
     nit: 29
 success: True
       x: -0.080272338684177602
# [-0.080272338684177602]  = value of delta_Mo that minimize the Hubble residual.



In [49]:
"""
     fun: 1.482459899373599e-12
    nfev: 33
     nit: 32
 success: True
       x: 5.0906660693874514e-07
# [5.0906660693874514e-07]  = value of delta_Mo that minimize the Hubble residual.
"""
0

0

In [50]:
# Add a value to the theoretical distance modulus so that the weighted 
# average of the Hubble residual is zero. This is only for the SALT2 Hubble diagram.

if BandMax == 'SALT2': # ORIGINAL
# if BandMax == 'SALT2' or BandMax == 'Snoopy': # FOR RAISINS OPTICAL FITS
    delta_Mo = SimplexResult_1[0] 

else: 
    delta_Mo_int1 = np.array([0.])
    delta_Mo = delta_Mo_int1[0]
    
print '# delta_Mo = %s'%delta_Mo
# print '# type of variable: %s'%type(delta_Mo)

# delta_Mo = 0.0


In [51]:
#-- Array of minimum redshift to consider --

# zCMB_Min = np.array([0, 0.01])
# zCMB_Min_Label = ['z0', 'z001']

zCMB_Min = np.array([0])
zCMB_Min_Label = ['z0']

#--------------------------
#   Color array
# ColorSamples = [True, False]
ColorSamples = [True]

### Plot Hubble diagram

In [52]:
# To display where the currently active matplotlibrc file was loaded from, 
# one can do the following:

# import matplotlib
#  matplotlib.matplotlib_fname()

In [53]:
# u'/Users/arturo/anaconda2/lib/python2.7/site-packages/matplotlib/mpl-data/matplotlibrc'

In [54]:
# import matplotlib
# matplotlib.__version__

In [55]:
fontsizePlot = 10.5

# To plot the theoretical (spectroscopic) distance modulus
nbins1= 51
z1 = np.linspace(xlimPlots[0], xlimPlots[1], nbins1)  
mu1 = DistanceMuVector(z1, OmMFix, wFix, HoFix) +delta_Mo # ORIGINAL
# FOR RAISINS SNOOPY FIT:
# mu1 = DistanceMuVector(z1, OmMFix, wFix, HoFix) -delta_Mo 

# Count number of SNe for each subsample
count_CfA = 0; count_CSP = 0; count_all = 0

#---------------------------------
# Plot

for k in range(len(ColorSamples)): # Loop over the colors
    
    for j in range(len(zCMB_Min)):  # Loop over the 'zCMB_Min' array

        #------- Plotting the data -------
        fig = plt.figure()
        # fig = plt.figure(figsize=(8,6), dpi=80)
        # OLD. fig = plt.figure(figsize=(12, 8))
        
        # loop over 'DistanceMu_Good_AfterCutoffs_Main_.txt'
        for i in range(len(DistMu_array)): 

            zzInt = DistMu_array[i][1] # z_CMB for a given SNe
            # chi2dofInt = DistMu_array[i][6] # chi2 by d.o.f.
            # residInt = abs(DistMu_array[i][5]) # absolute value of the residual value
            sampleFlag = DistMu_array[i][7]  # Subsample           
                
            # Create the variable "snName" containing the first 8 
            # (or 7) letters of the SNe file name
            # I use "snName" to compare with the SNe names in 
            # 'SNeWithCepheidDistances.txt', so that
            # I will not compute a peculiar velocity uncertainty for those SNe.
            try:
                if   DistMu_array[i][0][7] == '_': 
                    snName = DistMu_array[i][0][:7] # To read correctly, e.g., "sn2011B_"
                elif DistMu_array[i][0][7] != '_':
                    # To read correctly, e.g., "snf20080514-002":
                    if is_number(DistMu_array[i][0][7]): snName = DistMu_array[i][0][:15] 
                    else: snName = DistMu_array[i][0][:8]  # To read correctly, e.g., "sn1998bu"   
            except: snName = DistMu_array[i][0][:6]  # To read correctly, e.g., "sn2011B"
                
                
            # Make special symbol for SNe with Cepheid.
            if snName in ListSNeCepheid['f0']: fmt_int = '*'
            else: fmt_int = '.'
            
            if ColorSamples[k] and KindOfData4HD == 'AllSamples':
                # CSP data
                if sampleFlag == 2 and zzInt > zCMB_Min[j]:   
                    plt.errorbar(zzInt, DistMu_array[i][3], yerr=DistMu_array[i][4], 
                        fmt=fmt_int, color='blue', ms=MyPointSize, 
                        elinewidth=MyLineWidth, capsize=MyCapeSize)
                # CfA data
                elif sampleFlag == 1 and zzInt > zCMB_Min[j]: 
                    plt.errorbar(zzInt, DistMu_array[i][3], yerr=DistMu_array[i][4], 
                        fmt=fmt_int, color='red', ms=MyPointSize, 
                        elinewidth=MyLineWidth, capsize=MyCapeSize)
                # Others data
                elif sampleFlag == 3 and zzInt > zCMB_Min[j]: 
                    plt.errorbar(zzInt, DistMu_array[i][3], yerr=DistMu_array[i][4], 
                        fmt=fmt_int, color='green', ms=MyPointSize, 
                        elinewidth=MyLineWidth, capsize=MyCapeSize)
                    
            else:
                if zzInt > zCMB_Min[j]:  
                    if KindOfData4HD == 'CfA' and sampleFlag == 1:
                        plt.errorbar(zzInt, DistMu_array[i][3], yerr=DistMu_array[i][4], 
                            fmt=fmt_int, color='red', ms=MyPointSize, 
                            elinewidth=MyLineWidth, capsize=MyCapeSize)
                        # count_CfA = count_CfA + 1 # Counter
                    elif KindOfData4HD == 'CSP'  and sampleFlag == 2:
                        plt.errorbar(zzInt, DistMu_array[i][3], yerr=DistMu_array[i][4], 
                            fmt=fmt_int, color='blue', ms=MyPointSize, 
                            elinewidth=MyLineWidth, capsize=MyCapeSize)
                        # count_CSP = count_CSP + 1 # Counter
                    elif KindOfData4HD == 'Others'  and sampleFlag == 3:
                        plt.errorbar(zzInt, DistMu_array[i][3], yerr=DistMu_array[i][4], 
                            fmt=fmt_int, color='green', ms=MyPointSize, 
                            elinewidth=MyLineWidth, capsize=MyCapeSize)
                    elif KindOfData4HD == 'AllSamples':
                        plt.errorbar(zzInt, DistMu_array[i][3], yerr=DistMu_array[i][4], 
                            fmt=fmt_int, color='black', ms=MyPointSize, 
                            elinewidth=MyLineWidth, capsize=MyCapeSize)
                        # count_all = count_all + 1 # Counter    
             
        # Plotting the theory line
        plt.plot(z1, mu1, color='black')

        plt.xlim(xlimPlots)
        plt.ylim(ylimPlots)

        plt.xlabel('Redshift', fontsize=fontsizePlot)
        plt.ylabel(r'Distance modulus $\mu$', fontsize=fontsizePlot)
        
        #--- Plot Title ---
        if PlotTotalMu == False:
            plt.title('Hubble diagram (%s band)'%(BandName), fontsize=fontsizePlot+2)
        elif PlotTotalMu == True:
            if BandsCombination == 'AllBands':
                plt.title('Hubble diagram (all YJHK bands)', fontsize=fontsizePlot+2)
            else: plt.title('Hubble diagram (%s bands)'%(BandsCombination), fontsize=fontsizePlot+2)
        
        # plt.legend(loc='lower right')
        # plt.tick_params(labelsize=fontsizePlot)
        plt.tick_params(axis='x', labelsize=fontsizePlot)
        plt.tick_params(axis='y', labelsize=fontsizePlot+2)        

        plt.grid(True)
        plt.tight_layout()
                
        # if   PlotTotalMu == True:  
        #     NIRbandsText_1 = BandsCombination+'_'
        #     NIRbandsText_2 = BandsCombination+'/'
        # elif PlotTotalMu == False: 
        #     NIRbandsText_1 = ''; NIRbandsText_2 = ''
            
        if   PlotTotalMu == True:  NIRbandsText = BandsCombination+'_'
        elif PlotTotalMu == False: NIRbandsText = ''

        if ColorSamples[k] and KindOfData4HD == 'AllSamples':
            plt.savefig(DirSaveOutput+'Plot_HD_%s_%s_%s_%s_%s_%s_Color_%s.png'%(
                KindOfData4HD, KindOfTemp,
                              KindOfTempSubgroup, 
                                   chi2_dof_Max_Label, residualMax_Label, 
                                      zCMB_Min_Label[j], NIRbandsText)
                             , dpi=ResolutionPlot_HD, format='png'
                       )
        else:
            plt.savefig(DirSaveOutput+'Plot_HD_%s_%s_%s_%s_%s_%s_%s.png'%(
                KindOfData4HD, KindOfTemp,
                                  KindOfTempSubgroup,
                                    chi2_dof_Max_Label, residualMax_Label, 
                                       zCMB_Min_Label[j], NIRbandsText) 
                                        , dpi=ResolutionPlot_HD, format='png'
                        )
        plt.close()

# print 'Number SNe from CfA: ', count_CfA
# print 'Number SNe from CSP: ', count_CSP
# print 'Number SNe from all: ', count_all

In [56]:
# To display where the currently active matplotlibrc file was loaded from, one can do the following:

# import matplotlib
# matplotlib.matplotlib_fname()

## Intrinsic dispersion

In [57]:
# Creation of arrays for mu_residuals.

mu_resid_z0_np = np.zeros(len(DistMu_array))
sigma2_appMagTBmax_z0_np = np.zeros(len(DistMu_array))
sigma2_pec_z0_np = np.zeros(len(DistMu_array))

mu_resid_z001 = []
sigma2_appMagTBmax_z001 = []
sigma2_pec_z001 = []

for i in range(len(DistMu_array)): # loop over 'DistanceMu_Good_AfterCutoffs_Main_.txt'
    
    zcmbInt     = DistMu_array[i][1]  # zcmb
    err_zcmbInt = DistMu_array[i][2]  # error_zcmb
    mu_resid_z0_np[i]   = DistMu_array[i][5] +delta_Mo  # Delta_mu
    
    # Create the variable "snName" containing the first 8 (or 7) letters of the SNe file name
    # I use "snName" to compare with the SNe names in 'SNeWithCepheidDistances.txt', so that
    # I will not compute a peculiar velocity uncertainty for those SNe.
    try:
        if   DistMu_array[i][0][7] == '_': 
            snName = DistMu_array[i][0][:7]  # To read correctly, e.g., "sn2011B_"
        elif DistMu_array[i][0][7] != '_':
            # To read correctly, e.g., "snf20080514-002":
            if is_number(DistMu_array[i][0][7]): snName = DistMu_array[i][0][:15] 
            else: snName = DistMu_array[i][0][:8]  # To read correctly, e.g., "sn1998bu"  
    except: snName = DistMu_array[i][0][:6]  # To read correctly, e.g., "sn2011B"
    
    
    #--- Determine the uncertainty in distance modulus from the peculiar velocity uncertainty.---
    
    #-----------   For z > 0: --------------
    
    if PlotTotalMu == True: # For 'PlotTotalMu' *compute* "sigma_mu_pecVel"
        if snName in ListSNeCepheid['f0']: 
            sigma2_pec_z0_np[i] = sigma2_pec(zcmbInt, err_zcmbInt, 0)
        else: sigma2_pec_z0_np[i] = sigma2_pec(zcmbInt, err_zcmbInt, vpecFix)
    else: # For 'NIRmax', 'Bmax' 'Snoopy', 'SALT2' *reads* "sigma_mu_pecVel"
        sigma2_pec_z0_np[i] = (DistMu_array[i][11])**2   # (sigma_mu_pecVel)^2
    
    # Read the apparent-magnitude uncertainty at either T_Bmax or T_NIRmax:
    if PlotTotalMu==False and (BandMax == 'NIRmax' or 
                               BandMax == 'Bmax' or BandMax == 'Bmax_GP' or
                               BandMax == 'SALT2' or BandMax == 'Snoopy'):
        # error variance of the app mag at TBmax or TNIRmax:
        sigma2_appMagTBmax_z0_np[i] = (DistMu_array[i][9])**2  
    
    
    #-----------   For z > 0.01: --------------
    
    if zcmbInt > 0.01:
        mu_resid_z001 += [DistMu_array[i][5]]  # Delta_mu  
        
        if PlotTotalMu == True:
            if snName in ListSNeCepheid['f0']: 
                sigma2_pec_z001 += [sigma2_pec(zcmbInt, err_zcmbInt, 0)]
            else: sigma2_pec_z001 += [sigma2_pec(zcmbInt, err_zcmbInt, vpecFix)]
        else:  # For NIRmax', 'Bmax' 'Snoopy', 'SALT2' 
            sigma2_pec_z001 += [(DistMu_array[i][11])**2]  # (sigma_mu_pecVel)^2        
        
        # Read the apparent-magnitude uncertanty at either T_Bmax or T_NIRmax:
        if PlotTotalMu==False and (BandMax == 'NIRmax' or 
                                   BandMax == 'Bmax'  or BandMax == 'Bmax_GP' or
                                   BandMax == 'SALT2'  or BandMax == 'Snoopy'):
            # error variance of the app mag at TBmax or TNIRmax:
            sigma2_appMagTBmax_z001 += [(DistMu_array[i][9])**2]  
        
# Convert the list to np.arrays:
mu_resid_z001_np = np.array(mu_resid_z001)
sigma2_appMagTBmax_z001_np = np.array(sigma2_appMagTBmax_z001)
sigma2_pec_z001_np = np.array(sigma2_pec_z001)

In [58]:
# sigma2_appMagTBmax_z0_np

In [59]:
# sigma2_appMagTBmax_z001_np

In [60]:
# print 'Number of SNe with z>0 :', len(mu_resid_z0_np)
# print 'Number of SNe with z>0.01 :', len(mu_resid_z001_np)

In [61]:
# Checking the sized of the arrays: the left and right numbers have to be the same
# in a given row printed below.

print '#', len(mu_resid_z0_np),   len(sigma2_appMagTBmax_z0_np),   len(sigma2_pec_z0_np)
print '#', len(mu_resid_z001_np), len(sigma2_appMagTBmax_z001_np), len(sigma2_pec_z001_np)

# 119 119 119
# 95 95 95

# 63 63 63
# 49 49 49

# 29 29 29
# 26 26 26


In [62]:
# Finding the best estimated value for sigma2Pred by minimizing the -2ln(Likelihood) function.

# Defining the function to compute the intrinsic dispersion (sigmaPred) instead
# of the square of the intrinsic dispersion (sigma2Pred): for the case of determining
# the instrisic dispersion.
# I obtain an error due to a very small value of sigmaPred, so that during minimizing 
# the likelihood function to determine sigmaPred, it is sampled some negative values.

# Likelihood to determine the intrinsic dispersion
# -2ln(Likelihood) function, Eq. (B.6) of Blondin et al 2011

# For the case of using a normalized template. This is the full Eq. (B.6) of Blondin et al 2011
def neg2lnLikeFull(sigmaPred, mu_resid_np, sigma2_pec_np, sigma2_appmagTBmax_np):
    sum1 = 0
    for i in range(len(mu_resid_np)):
        sum1 = (sum1 + np.log(sigma2_appmagTBmax_np[i] + sigmaPred**2 + sigma2_pec_np[i]) +  
                (mu_resid_np[i]**2)/(sigma2_appmagTBmax_np[i] + sigmaPred**2 + sigma2_pec_np[i]) )  
    return sum1

#----

# The truncated version.
def neg2lnLike(sigmaPred, mu_resid_np, sigma2_pec_np):
    sum1 = 0
    for i in range(len(mu_resid_np)):
        sum1 = (sum1 + np.log(sigmaPred**2 + sigma2_pec_np[i]) + 
                (mu_resid_np[i]**2)/(sigmaPred**2 + sigma2_pec_np[i]) ) 
    return sum1

#-----------------------------------------------------------------
from scipy.optimize import fmin as simplex

InitialGuess = 0.15

if Method==7 and PlotTotalMu==False and (BandMax == 'NIRmax' or BandMax == 'Bmax' or BandMax == 'Bmax_GP' #):
                                         or BandMax == 'SALT2' or BandMax == 'Snoopy'): # ORIGINAL
                                         # or BandMax == 'SALT2'): # FOR RAISINS SNOOPY FIT
    SimplexResult_z0_2 = simplex(neg2lnLikeFull, InitialGuess, 
                                   args=(mu_resid_z0_np, sigma2_pec_z0_np, sigma2_appMagTBmax_z0_np))

    SimplexResult_z001_2 = simplex(neg2lnLikeFull, InitialGuess, 
                                   args=(mu_resid_z001_np, sigma2_pec_z001_np, sigma2_appMagTBmax_z001_np))

    
elif PlotTotalMu==True: # ORIGINAL
# elif PlotTotalMu==True or BandMax == 'Snoopy' or BandMax == 'SALT2': # FOR RAISINS SNOOPY FIT
    SimplexResult_z0_2 = simplex(neg2lnLike, InitialGuess, 
                              args=(mu_resid_z0_np,   sigma2_pec_z0_np))
    
    SimplexResult_z001_2 = simplex(neg2lnLike, InitialGuess, 
                              args=(mu_resid_z001_np, sigma2_pec_z001_np))

SimplexResult_z0 = SimplexResult_z0_2**2
SimplexResult_z001 = SimplexResult_z001_2**2

# print 'Best estimated value of sigma_Pred (z>0) =', SimplexResult_z0_2
# print 'Best estimated value of sigma_Pred (z>0.01) =', SimplexResult_z001_2

print 
print 'Best estimated value of sigma^2_Pred (z>0) =', SimplexResult_z0
print 'Best estimated value of sigma_Pred (z>0) =', SimplexResult_z0_2

print 
print 'Best estimated value of sigma^2_Pred (z>0.01) =', SimplexResult_z001
print 'Best estimated value of sigma_Pred (z>0.01) =', SimplexResult_z001_2


Optimization terminated successfully.
         Current function value: -91.622139
         Iterations: 13
         Function evaluations: 26
Optimization terminated successfully.
         Current function value: -86.810125
         Iterations: 13
         Function evaluations: 26

Best estimated value of sigma^2_Pred (z>0) = [ 0.00678693]
Best estimated value of sigma_Pred (z>0) = [ 0.08238281]

Best estimated value of sigma^2_Pred (z>0.01) = [ 0.00527893]
Best estimated value of sigma_Pred (z>0.01) = [ 0.07265625]


In [63]:
# Computing the uncertainty on sigma_pred

# Define the Fisher information matrix, Eq. (B.7) of Blondin et al 2011

# For the case of using a normalized template. This is the full Eq. (B.7) of Blondin et al 2011
def FisherFuncFull(sigmaPred, mu_resid_np, sigma2_pec_np, sigma2_appmagTBmax_np):
    sum2 = 0
    for i in range(len(mu_resid_np)):
        sum2 = (sum2 + (mu_resid_np[i]**2)/(sigma2_appmagTBmax_np[i] + sigmaPred**2 + sigma2_pec_np[i])**3 -
               1/(2*(sigma2_appmagTBmax_np[i] + sigmaPred**2 + sigma2_pec_np[i])**2)  )
    return sum2

#--------

def FisherFunc(sigmaPred, mu_resid_np, sigma2_pec_np):
    sum2 = 0
    for i in range(len(mu_resid_np)):
        sum2 = (sum2 + (mu_resid_np[i]**2)/(sigmaPred**2 + sigma2_pec_np[i])**3 -
               1/(2*(sigmaPred**2 + sigma2_pec_np[i])**2)  )
    return sum2

#-----------------------------------------------------------------

# The variance error of sigma^2_pred:
if Method==7 and PlotTotalMu==False and (BandMax == 'NIRmax' or BandMax == 'Bmax' or 
                                         BandMax == 'Bmax_GP' or
                                        BandMax == 'SALT2'  or BandMax == 'Snoopy'):
    Var_sigma2_pred_z0 =   1/FisherFuncFull(SimplexResult_z0_2[0],   
                                            mu_resid_z0_np,   sigma2_pec_z0_np, sigma2_appMagTBmax_z0_np)
    Var_sigma2_pred_z001 = 1/FisherFuncFull(SimplexResult_z001_2[0], 
                                            mu_resid_z001_np, sigma2_pec_z001_np, sigma2_appMagTBmax_z001_np)
    
elif PlotTotalMu==True:
    Var_sigma2_pred_z0 =   1/FisherFunc(SimplexResult_z0_2[0],   mu_resid_z0_np,   sigma2_pec_z0_np)
    Var_sigma2_pred_z001 = 1/FisherFunc(SimplexResult_z001_2[0], mu_resid_z001_np, sigma2_pec_z001_np)
    
error_sigma_pred_z0   = np.sqrt(Var_sigma2_pred_z0  /(4*SimplexResult_z0[0]))
error_sigma_pred_z001 = np.sqrt(Var_sigma2_pred_z001/(4*SimplexResult_z001[0]))

print 'Variance of sigma^2_pred (z>0) =', round(Var_sigma2_pred_z0,6)
print 'Variance of sigma^2_pred (z>0.01) =', round(Var_sigma2_pred_z001,6)
print '\nError_sigma_pred (z>0) =', round(error_sigma_pred_z0,3)
print 'Error_sigma_pred (z>0.01) =', round(error_sigma_pred_z001,3)


Variance of sigma^2_pred (z>0) = 1.3e-05
Variance of sigma^2_pred (z>0.01) = 1e-05

Error_sigma_pred (z>0) = 0.022
Error_sigma_pred (z>0.01) = 0.022


#### Report the intrinsic dispersion

In [64]:
print '#'+'-'*50
# print "# %s band, vpecFix = %s km/s"%(Band, vpecFix)
# print "# Intrinsic dispersion for the case (0 < z < %s):"%(zcmb_Max)
print '# Intrinsic dispersion = %s +/- %s'%(np.sqrt(SimplexResult_z0[0]), error_sigma_pred_z0)

# ------------------------------
now = datetime.datetime.now() # Read the time and date right now
text_timenow = now.strftime("%Y-%m-%d; %H:%M hrs.")
text_Date   = '# On date: %s \n'%text_timenow
# ------------------------------

print ' '
print '# %s | Date: %s '%(AppMag_method, text_timenow)
print "# IntrinsicDisp = %.5f # %s | vpecFix=%s km/s | 0<z<%s."%(
    np.sqrt(SimplexResult_z0[0]), BandName, vpecFix, zcmbUpperLim)

#--------------------------------------------------
# Intrinsic dispersion = 0.0823828125 +/- 0.0215651882723
 
# Gaussian Process to compute app mags | Date: 2018-07-23; 00:17 hrs. 
# IntrinsicDisp = 0.08238 # Y | vpecFix=150 km/s | 0<z<0.04.


In [65]:
#--------------------------------------------------
# Intrinsic dispersion = 0.07517578125 +/- 0.0629356479937
 
# Gaussian Process to compute app mags | Date: 2018-05-02; 17:04 hrs. 
# IntrinsicDisp = 0.07518 # K | vpecFix=150 km/s | 0<z<0.04.

#--------------------------------------------------
# Intrinsic dispersion = 0.12017578125 +/- 0.016725821387
 
# Gaussian Process to compute app mags | Date: 2018-02-22; 16:49 hrs. 
# IntrinsicDisp = 0.12018 # J | vpecFix=150 km/s | 0<z<0.04.

-------

# $\chi^2_{\rm d.o.f.}$

#### Checking the consistency between the error bars of the residual distance modulus vs the scatter in the Hubble-diagram residual plot

In [66]:
ratio_int = 0;  

for i in range(len(DistMu_array)):
    # print i
    
    mu_resid  = DistMu_array[i][5] +delta_Mo
    
    if   BandMax == 'Bmax':   IntrinsicDisp_int = np.sqrt(SimplexResult_z0[0])
    elif BandMax == 'NIRmax': IntrinsicDisp_int = np.sqrt(SimplexResult_z0[0])
    elif BandMax == 'Bmax_GP': IntrinsicDisp_int = np.sqrt(SimplexResult_z0[0])
    elif BandMax == 'SALT2':  IntrinsicDisp_int = np.sqrt(SimplexResult_z0[0])
    elif BandMax == 'Snoopy': IntrinsicDisp_int = np.sqrt(SimplexResult_z0[0])
     
    if  PlotTotalMu == False and (BandMax == 'NIRmax' or BandMax == 'Bmax' or 
                                  BandMax == 'Bmax_GP' or
                                  BandMax == 'SALT2'  or BandMax == 'Snoopy'): 
        sigma_pecVel = DistMu_array[i][11]
        error_appMag = DistMu_array[i][9]
        ratio_int = ratio_int + ((mu_resid**2) / (error_appMag**2 + 
                                                  sigma_pecVel**2 + 
                                                  IntrinsicDisp_int**2) )
        
        # ratio_int = ratio_int + ((mu_resid**2) / (error_appMag**2 + 
        #                                         sigma_pecVel**2 ) )
    
    elif PlotTotalMu == True:  
        sigma_pecVel = np.sqrt(sigma2_pec(zcmbInt, err_zcmbInt, vpecFix))
        error_mu = DistMu_array[i][4]   # error_mu
        ratio_int = ratio_int + ((mu_resid**2) / (error_mu**2 + sigma_pecVel**2) )  
    
chi2_dof_HD    = ratio_int / len(DistMu_array)


print '#'+'-'*40
print "# %s band | vpecFix = %s km/s | 0 < z < %s"%(BandName, vpecFix, zcmbUpperLim)
print '# %s.'%AppMag_method
print '# chi^2 =', ratio_int, '| Number of data =', len(DistMu_array)
print '# chi^2_dof =', chi2_dof_HD

#----------------------------------------
# Y band | vpecFix = 150 km/s | 0 < z < 0.04
# Gaussian Process to compute app mags.
# chi^2 = 31.2389298696 | Number of data = 29
# chi^2_dof = 1.07720447826


## Write the summary of values to a text file

In [67]:
if (BandMax != 'NIRmax' or PlotTotalMu == True):
    textfile_1 = open(DirSaveOutput+"Summary_HDScatter_RMS_.txt", 'w')

    now = datetime.datetime.now() # Read the time and date right now
    text_timenow = now.strftime("%Y-%m-%d (yyyy-mm-dd).")
    text_Author = '# Data table created by: Arturo Avelino \n'
    text_Date   = '# On date: %s \n'%text_timenow
    text_script = '# Script used: %s \n'%NotebookName
    text_line = '#'+'-'*50 + '\n'

    textfile_1.write("# Summary of the scatter in the Hubble residual \n")
    textfile_1.write("# %s band \n"%BandName)

    textfile_1.write(text_line)
    textfile_1.write(text_Author); textfile_1.write(text_Date); 
    textfile_1.write(text_script); 
    textfile_1.write(text_line)

    if (BandMax == 'Bmax' and PlotTotalMu == False):
        
        try: 
            # This section is for the case that I've used 11.ipynb to compute
            # the distance moduli, so that the values above were previously defined
            # during the creation of the "DistanceMu_Good_AfterCutoffs_Main_.txt"
            # text file.
            textfile_1.write(text01); textfile_1.write(text1); textfile_1.write(text2);
            textfile_1.write(text3); textfile_1.write(text3_0_1); textfile_1.write(text3_3);
            textfile_1.write(text3_4); textfile_1.write(text3_5); 
            textfile_1.write(text_line)
            
        except:
            # This section is for the case that I'm using 11.ipynb just to plot.
            text01 = '# Apparent mag data used to construct the Hubble diagram: %s \n'%KindOfData4HD
            text1 = '# Template used: \n'
            text2 = '# %s \n'%DirTemplate[78:]
            text3 = '# Cosmology used to compute residuals: (Omega_M=%r, Omega_L=%r, w=%r, Ho=%r) \n'%(OmMFix, OmLFix, wFix, HoFix)

            text3_0_0 = '# 0.01 < z_cmb < %r \n'%zcmbUpperLim
            text3_0_1 = '# Cutoffs: z_cmb<%r | %r<dm15<%r | %r<EBVhost<%r | EBV_mw<%r | chi2_dof<%r | Residual<%r \n'%(zcmbUpperLim,
                dm15LowerLim,  dm15UpperLim, EBVhostMin, EBVhostMax, EBVMWLim, chi2_dof_Max, residualMax)
            text3_3 = '# Phase range used of the template: (%r, %r) days \n'%(PhaseMinTemp, PhaseMaxTemp)
            text3_4 = '# Minimal number of data in LC to be considered for the Hubble diagram: %r \n'%MinNumOfDataInLC
            text3_5 = '# Assumed uncertainty in the peculiar velocity: %r km/s \n'%vpecFix

            textfile_1.write(text01); textfile_1.write(text1); textfile_1.write(text2);
            textfile_1.write(text3); textfile_1.write(text3_0_1); textfile_1.write(text3_3);
            textfile_1.write(text3_4); textfile_1.write(text3_5); 
            textfile_1.write(text_line)
    
    textfile_1.write('# %.5f # = delta_Mo. Valued ADDED to the theoretical distance \n\
# modulus so that the weighted average of the Hubble residual is zero. \n'%delta_Mo)
        
    textfile_1.write('# Intrinsic dispersion for the case (0 < z < %s) and used to obtain the \n\
# total photometric distance modulus uncertainty: \n'%zcmbUpperLim)
    
    textfile_1.write('%14.10f  %.10f  # Intrinsic dispersion and its uncertainty for the case \
0 < z_cmb < %s \n'%(np.sqrt(SimplexResult_z0[0]), error_sigma_pred_z0, zcmbUpperLim))
    textfile_1.write('%14.10f  %.10f  # Intrinsic dispersion and its uncertainty for the case \
0.01 < z_cmb < %s \n'%(np.sqrt(SimplexResult_z001[0]), error_sigma_pred_z001, zcmbUpperLim))

    textfile_1.write('%14.10f  %.10f  # (AverageAbsMag_atMax, err_AverageAbsMag_atMax) \
    \n'%(Average_NIRAbsMag_TBmax, error_Average_NIRAbsMag_TBmax))

    textfile_1.write('%14.10f  %-12.0f  # (chi^2, Number of SNe) for the case (0 < z < %s) \
    \n'%(ratio_int, len(DistMu_array), zcmbUpperLim))
    textfile_1.write('%14.10f  0             # chi^2_dof \n'%chi2_dof_HD)

    textfile_1.write(text_line)
    textfile_1.close()

In [68]:
# textfile_1.close();

# Plot residual

In [72]:
# Plot error bars of the Hubble residuals?
# This option sometimes is useful to better visualize any trend
# in the Hubble residual plot. 
# NOTE: This only apply to Y-band data, I need to implement the
# instruction in the other NIR bands.
# plotErrorBars_list = [True, False]
plotErrorBars_list = [True]

fontsizePlot = 10.5

# Label the outliers.
# In one plot, print the name of those SNe with one value in "label_array"
# and in another plot, print those SNe with the other value.
label_array = np.array([0.23, 3])

#---------------------------------
#    Text files

if   PlotTotalMu == False: textfile7 = open(DirSaveOutput+'Verbose_%s.txt'%BandName, 'w')
elif PlotTotalMu == True:  textfile7 = open(DirSaveOutput+'Verbose_%s.txt'%BandsCombination, 'w')   

# Append data to the already existing file 'Summary_HDScatter_.txt', but
# if the file doesn't exist then create it:
textfile_8 = open(DirSaveOutput+'Summary_HDScatter_RMS_.txt', 'a')

textfile7.write('# RMS SUMMARY \n')
textfile7.write(' \n')

now = datetime.datetime.now() # Read the time and date right now
text_timenow = now.strftime("%Y-%m-%d (yyyy-mm-dd)")
text_line = '#'+'-'*50 + '\n'

# Add a value to the theoretical distance modulus so that the weighted 
# average of the Hubble residual is zero. This is only for the SALT2 Hubble diagram.
textfile7.write('%s # = delta_Mo. Valued ADDED to the theoretical distance modulus so that \
the weighted average of the Hubble residual is zero: \n'%delta_Mo)

textfile7.write('%s band | vpecFix = %s km/s \n'%(BandName, vpecFix))
textfile7.write(' \n')

textfile7.write('intrinsic dispersion sigma_Pred (z>0): %r +/- %r \n'%
                (np.sqrt(SimplexResult_z0[0]), error_sigma_pred_z0))
textfile7.write('intrinsic dispersion sigma_Pred (z>0.01): %r +/- %r \n'%
                (np.sqrt(SimplexResult_z001[0]), error_sigma_pred_z001))

textfile7.write('\n Checking the consistency between the error bars of the residual \
distance modulus vs the scatter in the Hubble-diagram residual plot (case 0 < z < %s) \n'%zcmbUpperLim)
textfile7.write('chi^2 = %s | Number of data = %s \n'%(ratio_int, len(DistMu_array)))
textfile7.write('chi^2_dof = %s \n'%chi2_dof_HD)
textfile7.write(text_line)

textfile7.write('# Data table created by: Arturo Avelino \n')
textfile7.write('# On date: %s \n'%text_timenow)
textfile7.write('# Script used: %s \n'%NotebookName)
textfile7.write(text_line)

#---------------------------------

#    Theoretical values

# Array plot the -theoretical- (spectroscopic) distance modulus 
nbins1= 151
# old. z1 = np.linspace(xlimPlots[0], xlimPlots[1], nbins1)
z1 = np.linspace(xlimPlots[0], xlimPlots[1]+0.05, nbins1)
mu0 = np.zeros(len(z1))

# Array to plot the -theoretical- peculiar velocity uncertanty
# old. error_z1 = 0.001 * np.ones(len(z1)) # 0.001 is the average error_zcmb in Andy's compilation
error_z1 = err_zCMB_fix * np.ones(len(z1))
sigma_pec_np = np.sqrt(sigma2_pec(z1, error_z1, vpecFix))


#---------------------------------
# Plot

# Loop over plotting or not the error bars.
for i3 in range(len(plotErrorBars_list)):
    
    plotErrorBars = plotErrorBars_list[i3]

    for ll in label_array: # Loop to label the name of the outlier SNe
        labelOutlierLim = ll # Label the outliers with residual >= this quantity 

        for k in range(len(ColorSamples)): # Loop over the colors

            for j in range(len(zCMB_Min)):  # Loop over the 'zCMB_Min' array

                # fig = plt.figure(figsize=(12, 8))
                fig = plt.figure()

                # Plotting theory: Flat Universe
                plt.plot(z1, mu0, color='black', lw=2, alpha=0.5)

                # Plot the peculiar velocity uncertainty

                plt.plot(z1, sigma_pec_np, ls='--', color='black')
                plt.plot(z1, -sigma_pec_np, ls='--', color='black') 


                # Reset all the values
                # To be used to compute the RMS:
                mu_resid_all = []; error_mu_all = []; sigma2_pec_all=[];
                mu_resid_CSP = []; error_mu_CSP = []; sigma2_pec_CSP=[]; countCSP = 0;
                mu_resid_CfA = []; error_mu_CfA = []; sigma2_pec_CfA=[]; countCfA = 0;
                mu_resid_Others = []; error_mu_Others = []; sigma2_pec_Others=[]; countOthers = 0

                mu_resid_all_np = 0;  error_mu_all_np = 0; sigma2_pec_all_np=0;
                mu_resid_CSP_np = 0;  error_mu_CSP_np = 0; sigma2_pec_CSP_np=0;
                mu_resid_CfA_np = 0;  error_mu_CfA_np = 0; sigma2_pec_CfA_np=0;
                mu_resid_Others_np = 0;  error_mu_Others_np = 0; sigma2_pec_Others_np=0;

                rms = 0;     ratio = 0;     rms_weight =0;
                rms_CfA = 0; ratio_CfA = 0; rms_CfA_weight =0;
                rms_CSP = 0; ratio_CSP = 0; rms_CSP_weight =0;
                rms_Others = 0; ratio_Others = 0; rms_Others_weight =0;

                #----  Plotting the data ----

                for i in range(len(DistMu_array)): # loop over 'DistanceMu_Good_AfterCutoffs_Main_.txt'

                    zzInt = DistMu_array[i][1] # z_CMB for a given SNe
                    err_zz= DistMu_array[i][2] # z_CMB for a given SNe
                    residInt = abs(DistMu_array[i][5]) # residual (absolute value)
                    error_mu = DistMu_array[i][4] # uncertainty in the distance modulus, mu.
                    mu_resid_int = DistMu_array[i][5]+delta_Mo # Delta_mu  

                    # Create the variable "snName" containing the first 8 (or 7) letters of the SNe file name
                    # I use "snName" to compare with the SNe names in 'SNeWithCepheidDistances.txt', so that
                    # I will not compute a peculiar velocity uncertainty for those SNe.

                    # ORIGINAL:
                    try:
                        if   DistMu_array[i][0][7] == '_': 
                            snName = DistMu_array[i][0][:7] # To read correctly, e.g., "sn2011B_"
                        elif DistMu_array[i][0][7] != '_':
                            # To read correctly, e.g., "snf20080514-002":
                            if is_number(DistMu_array[i][0][7]): snName = DistMu_array[i][0][:15] 
                            else: snName = DistMu_array[i][0][:8]  # To read correctly, e.g., "sn1998bu"   
                    except: snName = DistMu_array[i][0][:6] # To read correctly, e.g., "sn2011B"

                    # FOR RAISINS SNOOPY
                    # snName = DistMu_array[i][0]

                    # ORIGINAL
                    SNname = DistMu_array[i][0][2:8]
                    # FOR RAISINS SNOOPY
                    # SNname = DistMu_array[i][0]

                    if PlotTotalMu == True:
                        if snName in ListSNeCepheid['f0']: 
                            sigma2_pec_int = sigma2_pec(zzInt, err_zz, 0)
                        else: sigma2_pec_int = sigma2_pec(zzInt, err_zz, vpecFix)
                    else:
                        sigma2_pec_int = (DistMu_array[i][11])**2  # (sigma_mu_pecVel)^2            

                    sampleFlag = DistMu_array[i][7] # Distinguish between CfA or CSP


                    # Make special symbol for SNe with Cepheid.
                    if snName in ListSNeCepheid['f0']: fmt_int = '*'
                    else: fmt_int = '.'

                    #====================================================

                    if ColorSamples[k] and KindOfData4HD == 'AllSamples':
                        #--- CSP data:
                        if sampleFlag == 2 and zzInt > zCMB_Min[j]: 
                            if plotErrorBars:
                                plt.errorbar(zzInt, mu_resid_int, yerr=error_mu, 
                                    fmt=fmt_int, color='blue', ms=MyPointSize, 
                                    elinewidth=MyLineWidth, capsize=MyCapeSize)
                            else:
                                plt.plot(zzInt, mu_resid_int,
                                    marker=fmt_int, color='blue', ms=MyPointSize)

                            mu_resid_CSP += [mu_resid_int]
                            error_mu_CSP += [error_mu]
                            sigma2_pec_CSP += [sigma2_pec_int]
                            countCSP = countCSP + 1

                            # Label the SNe with residual larger than a given value.
                            if residInt > labelOutlierLim:
                                plt.text(zzInt+0.0005, mu_resid_int-0.015, SNname, 
                                         fontsize=(fontsizePlot-4), color='blue')

                        #--- CfA data
                        elif sampleFlag == 1 and zzInt > zCMB_Min[j]: 
                            plt.errorbar(zzInt, mu_resid_int, yerr=error_mu,
                                fmt=fmt_int, color='red',  ms=MyPointSize, 
                                elinewidth=MyLineWidth, capsize=MyCapeSize)
                            mu_resid_CfA += [mu_resid_int]
                            error_mu_CfA += [error_mu]
                            sigma2_pec_CfA += [sigma2_pec_int]
                            countCfA = countCfA + 1

                            # Label the SNe with residual larger than a given value.
                            if residInt > labelOutlierLim: 
                                plt.text(zzInt+0.0005, mu_resid_int-0.015, SNname, 
                                         fontsize=(fontsizePlot-4), color='red')

                        #--- Others data
                        elif sampleFlag == 3 and zzInt > zCMB_Min[j]: 
                            plt.errorbar(zzInt, mu_resid_int, yerr=error_mu, 
                                fmt=fmt_int, color='green', ms=MyPointSize, 
                                elinewidth=MyLineWidth, capsize=MyCapeSize)
                            mu_resid_Others += [mu_resid_int]
                            error_mu_Others += [error_mu]
                            sigma2_pec_Others += [sigma2_pec_int]
                            countOthers = countOthers + 1

                            # Label the SNe with residual larger than a given value.
                            if residInt > labelOutlierLim: 
                                plt.text(zzInt+0.0005, mu_resid_int-0.015, SNname, 
                                         fontsize=(fontsizePlot-4), color='green')

                    else:
                        if  zzInt > zCMB_Min[j]: 

                            if KindOfData4HD == 'CfA' and sampleFlag == 1:
                                plt.errorbar(zzInt, mu_resid_int, yerr=error_mu, 
                                    fmt=fmt_int, color='red', ms=MyPointSize, 
                                elinewidth=MyLineWidth, capsize=MyCapeSize)
                                mu_resid_all += [mu_resid_int]
                                error_mu_all += [error_mu]
                                sigma2_pec_all += [sigma2_pec_int]
                                # Label the SNe with residual larger than a given value.
                                if residInt > labelOutlierLim: 
                                    plt.text(zzInt+0.0005, mu_resid_int-0.015, SNname, 
                                             fontsize=(fontsizePlot-4), color='red')                        

                            elif KindOfData4HD == 'CSP'  and sampleFlag == 2:
                                plt.errorbar(zzInt, mu_resid_int, yerr=error_mu, 
                                    fmt=fmt_int, color='blue', ms=MyPointSize, 
                                    elinewidth=MyLineWidth, capsize=MyCapeSize)
                                mu_resid_all += [mu_resid_int]
                                error_mu_all += [error_mu]
                                sigma2_pec_all += [sigma2_pec_int]
                                # Label the SNe with residual larger than a given value.
                                if residInt > labelOutlierLim: 
                                    plt.text(zzInt+0.0005, mu_resid_int-0.015, SNname, 
                                             fontsize=(fontsizePlot-4), color='blue') 

                            elif KindOfData4HD == 'Others'  and sampleFlag == 3:
                                plt.errorbar(zzInt, mu_resid_int, yerr=error_mu, 
                                    fmt=fmt_int, color='green', ms=MyPointSize, 
                                    elinewidth=MyLineWidth, capsize=MyCapeSize)
                                mu_resid_all += [mu_resid_int]
                                error_mu_all += [error_mu]
                                sigma2_pec_all += [sigma2_pec_int]
                                # Label the SNe with residual larger than a given value.
                                if residInt > labelOutlierLim: 
                                    plt.text(zzInt+0.0005, mu_resid_int-0.015, SNname, 
                                             fontsize=(fontsizePlot-4), color='green') 

                            elif KindOfData4HD == 'AllSamples':
                                plt.errorbar(zzInt, mu_resid_int, yerr=error_mu, 
                                    fmt=fmt_int, color='black', ms=MyPointSize, 
                                    elinewidth=MyLineWidth, capsize=MyCapeSize)
                                mu_resid_all += [mu_resid_int]
                                error_mu_all += [error_mu]
                                sigma2_pec_all += [sigma2_pec_int]
                                # Label the SNe with residual larger than a given value.
                                if residInt > labelOutlierLim: 
                                    plt.text(zzInt+0.0005, mu_resid_int-0.015, SNname, 
                                             fontsize=(fontsizePlot-4), color='black')                             

                if ColorSamples[k] and KindOfData4HD == 'AllSamples':  
                    mu_resid_all = mu_resid_CSP + mu_resid_CfA + mu_resid_Others  
                    error_mu_all = error_mu_CSP + error_mu_CfA + error_mu_Others
                    sigma2_pec_all = sigma2_pec_CSP + sigma2_pec_CfA + sigma2_pec_Others

                #----- Computing the (RMS, WRMS) residuals --------

                mu_resid_all_np = np.array(mu_resid_all)
                error_mu_all_np = np.array(error_mu_all)
                sigma2_pec_all_np = np.array(sigma2_pec_all)
                if len(mu_resid_all) > 0:
                    rms = np.sqrt(np.mean(np.square(mu_resid_all)))
                    ratio = np.square(mu_resid_all_np)/(np.square(error_mu_all_np)+
                                                        sigma2_pec_all_np)
                    rms_weight = np.sqrt(np.sum(ratio)/np.sum(1/(np.square(error_mu_all_np)+
                                                                 sigma2_pec_all_np)))
                    textfile7.write('RMS all (z > %r) = %r (%r WRMS) \n'%(zCMB_Min[j], 
                                            round(rms,5), round(rms_weight,5)))
                    textfile7.write('   \n')

                    if ll == label_array[0] and k == 0:
                        textfile_8.write("%.10f  %.10f # (wRMS, RMS) for all. Case: z > %s.\n"%
                                     (rms_weight, rms, zCMB_Min[j]))
                    print 'RMS (z > %r, color=%s): %r (%r WRMS)'%(zCMB_Min[j], 
                            ColorSamples[k], round(rms,5), round(rms_weight,5))
                    print ' '

                mu_resid_CSP_np = np.array(mu_resid_CSP) 
                error_mu_CSP_np = np.array(error_mu_CSP)
                sigma2_pec_CSP_np = np.array(sigma2_pec_CSP)
                if len(mu_resid_CSP) > 0:
                    rms_CSP = np.sqrt(np.mean(np.square(mu_resid_CSP)))
                    ratio_CSP = np.square(mu_resid_CSP_np)/(np.square(error_mu_CSP_np)+
                                                            sigma2_pec_CSP_np)
                    rms_CSP_weight = np.sqrt(np.sum(ratio_CSP)/np.sum(1/(np.square(error_mu_CSP_np)+
                                                                         sigma2_pec_CSP_np)))
                    textfile7.write('RMS CSP (z > %r) = %r (%r WRMS) \n'%(zCMB_Min[j], 
                                            round(rms_CSP,5), round(rms_CSP_weight, 5)))
                    textfile7.write('   \n')
                    if ll == label_array[0] and k == 0:
                        textfile_8.write("%.10f  %.10f # (wRMS, RMS) for CSP subsample. Case: z > %s.\n"%
                                     (rms_CSP_weight, rms_CSP, zCMB_Min[j]))
                    print 'RMS CSP (z > %r, color=%s): %r (%r WRMS)'%(zCMB_Min[j], 
                            ColorSamples[k], round(rms_CSP,5), round(rms_CSP_weight, 5))
                    print ' '

                mu_resid_CfA_np = np.array(mu_resid_CfA)
                error_mu_CfA_np = np.array(error_mu_CfA)
                sigma2_pec_CfA_np = np.array(sigma2_pec_CfA)
                if len(mu_resid_CfA) > 0:
                    rms_CfA = np.sqrt(np.mean(np.square(mu_resid_CfA)))
                    ratio_CfA = np.square(mu_resid_CfA_np)/(np.square(error_mu_CfA_np)+
                                                            sigma2_pec_CfA_np)
                    rms_CfA_weight = np.sqrt(np.sum(ratio_CfA)/np.sum(1/(np.square(error_mu_CfA_np)+
                                                                         sigma2_pec_CfA_np)))
                    textfile7.write('RMS CfA (z > %r) = %r (%r WRMS) \n'%(zCMB_Min[j], 
                                            round(rms_CfA,5), round(rms_CfA_weight,5)))
                    textfile7.write('   \n')
                    if ll == label_array[0] and k == 0:
                        textfile_8.write("%.10f  %.10f # (wRMS, RMS) for CfA subsample. Case: z > %s.\n"%
                                     (rms_CfA_weight, rms_CfA, zCMB_Min[j]))
                    print 'RMS CfA (z > %r, color=%s): %r (%r WRMS)'%(zCMB_Min[j], 
                            ColorSamples[k], round(rms_CfA,5), round(rms_CfA_weight,5))

                mu_resid_Others_np = np.array(mu_resid_Others)
                error_mu_Others_np = np.array(error_mu_Others)
                sigma2_pec_Others_np = np.array(sigma2_pec_Others)
                if len(mu_resid_Others) > 0:
                    rms_Others = np.sqrt(np.mean(np.square(mu_resid_Others)))
                    ratio_Others = np.square(mu_resid_Others_np)/(np.square(error_mu_Others_np)+
                                                                  sigma2_pec_Others_np)
                    rms_Others_weight = np.sqrt(np.sum(ratio_Others)/np.sum(1/(np.square(error_mu_Others_np)+
                                                                               sigma2_pec_Others_np)))
                    textfile7.write('RMS Others (z > %r) = %r (%r WRMS)  \n'%(zCMB_Min[j], 
                                            round(rms_Others,5), round(rms_Others_weight,5)))
                    textfile7.write('   \n')
                    if ll == label_array[0] and k == 0:
                        textfile_8.write("%.10f  %.10f # (wRMS, RMS) for Others subsamples. Case: z > %s.\n"%
                                     (rms_Others_weight, rms_Others, zCMB_Min[j]))
                    print 'RMS Others (z > %r, color=%s): %r (%r WRMS)'%(zCMB_Min[j], 
                            ColorSamples[k], round(rms_Others,5), round(rms_Others_weight,5))

                #-----------------------------------------------------

                if plot_raisins == True: plt.xlim(0.2, 0.65) 
                else: plt.xlim(xlimPlots) 
                plt.ylim(ylimPlots_residual)

                #--- Labeling ----------------------------------------

                # PRINTING THE WEIGHTED RMSs AND INTRINSIC DISPERSION IN THE PLOTS.

                RightPlotLimitText_1 = 0.0135 # old: 0.05, 0.057
                if WRMS_label == True and j==0:     
                    plt.text(xlimPlots[0]+RightPlotLimitText_1-0.005, ylimPlots_residual[1]-0.15, 
                             r'%r SN, RMS=%.2f (%.2f WRMS), $\sigma_{\rm int}$=%.3f$\pm$%.3f'%(len(mu_resid_all),
                            rms, rms_weight, np.sqrt(SimplexResult_z0), error_sigma_pred_z0), 
                             fontsize=fontsizePlot-2)  

                elif WRMS_label == True and j==1:
                    plt.text(xlimPlots[0]+RightPlotLimitText_1-0.005, ylimPlots_residual[1]-0.15, 
                             r'%r SN, RMS=%.2f (%.2f WRMS), $\sigma_{\rm int}$=%.3f$\pm$%.3f'%(len(mu_resid_all),
                            rms, rms_weight, np.sqrt(SimplexResult_z001), error_sigma_pred_z001), 
                             fontsize=fontsizePlot-2)                              


                RightPlotLimitText_2 = 0.0235 # old: 0.04, 0.047
                if WRMS_label == True and WRMS_subsamples==True and ColorSamples[k] and KindOfData4HD == 'AllSamples':  
                    if plot_raisins == True:
                        plt.text(xlimPlots[0]+RightPlotLimitText_2, ylimPlots_residual[1]-0.25, 
                                 '%r DES, RMS=%.2f (%.2f WRMS)'%(countCfA, rms_CfA,rms_CfA_weight),
                                 color='red', fontsize=fontsizePlot-4)
                        plt.text(xlimPlots[0]+RightPlotLimitText_2, ylimPlots_residual[1]-0.35, 
                                 '%r PS1, RMS=%.2f (%.2f WRMS)'%(countCSP, rms_CSP, rms_CSP_weight), 
                                 color='blue', fontsize=fontsizePlot-4)
                        plt.text(xlimPlots[0]+(RightPlotLimitText_2-0.001), ylimPlots_residual[1]-0.45, 
                                 '%r Others, RMS=%.2f (%.2f WRMS)'%(countOthers, rms_Others,rms_Others_weight), 
                                 color='green', fontsize=fontsizePlot-4)
                    else: # plot the low-z:
                        plt.text(xlimPlots[0]+RightPlotLimitText_2, ylimPlots_residual[1]-0.25, 
                                 '%r CfA, RMS=%.2f (%.2f WRMS)'%(countCfA, rms_CfA,rms_CfA_weight), 
                                 color='red', fontsize=fontsizePlot-4)
                        plt.text(xlimPlots[0]+RightPlotLimitText_2, ylimPlots_residual[1]-0.35, 
                                 '%r CSP, RMS=%.2f (%.2f WRMS)'%(countCSP, rms_CSP, rms_CSP_weight), 
                                 color='blue', fontsize=fontsizePlot-4)
                        plt.text(xlimPlots[0]+(RightPlotLimitText_2-0.001), ylimPlots_residual[1]-0.45, 
                                 '%r Others, RMS=%.2f (%.2f WRMS)'%(countOthers, rms_Others,rms_Others_weight), 
                                 color='green', fontsize=fontsizePlot-4) 


                RightPlotLimitText_3 = 0.0235  # old: 0.032  
                # -NO- PRINTING THE WEIGHTED RMSs IN THE PLOTS.
                if WRMS_label == False and RMS_simple==True:

                    plt.text(xlimPlots[0]+RightPlotLimitText_3, ylimPlots_residual[1]- 0.15, # 0.12, 
                             '%r SN, RMS=%r'%(len(mu_resid_all),
                            round(rms,3)), fontsize=fontsizePlot-4)

                    if ColorSamples[k] and KindOfData4HD == 'AllSamples':
                        plt.text(xlimPlots[0]+RightPlotLimitText_3, ylimPlots_residual[1]- 0.25, # 0.20, 
                                 '%r CfA, RMS=%r'%(countCfA, round(rms_CfA,3)), 
                                 color='red', fontsize=fontsizePlot-4)
                        plt.text(xlimPlots[0]+RightPlotLimitText_3, ylimPlots_residual[1]- 0.35, # 0.28, 
                                 '%r CSP, RMS=%r'%(countCSP, round(rms_CSP,3)), 
                                 color='blue', fontsize=fontsizePlot-4)
                        plt.text(xlimPlots[0]+RightPlotLimitText_3, ylimPlots_residual[1]- 0.45, # 0.36, 
                                 '%r Others, RMS=%r'%(countOthers, round(rms_Others,3)), 
                                 color='green', fontsize=fontsizePlot-4)

                #-----------------------------------------------

                #--- Print the peculiar velocity, method, chi2_dof: -------

                # Peculiar velocity uncertainty
                plt.text(xlimPlots[0]+RightPlotLimitText_2, ylimPlots_residual[0]+0.35, 
                         r'$\sigma_{\rm pec}$ = %r km/s'%(vpecFix), color='black', 
                         fontsize=fontsizePlot-3)

                # chi2_dof: Consistency between error bars and scatter in the data
                if zCMB_Min[j] == 0:
                    plt.text(xlimPlots[0]+RightPlotLimitText_2, ylimPlots_residual[0]+0.25, 
                         r'$\chi^2_{\rm d.o.f.}$ = %.2f'%chi2_dof_HD, color='black', 
                             fontsize=fontsizePlot-3)

                # Method to determine distance moduli
                plt.text(xlimPlots[0]+RightPlotLimitText_2-0.01, ylimPlots_residual[0]+0.15, 
                         r'%s'%AppMag_method, color='black', fontsize=fontsizePlot-3)

                #---- Label title and axes ------

                if   BandMax == 'NIRmax': BandMaxText = '$T_{%s}$ max'%BandName 
                elif BandMax == 'Bmax' or BandMax == 'Bmax_GP':   BandMaxText = '$T_B$ max'
                elif BandMax == 'Snoopy': BandMaxText = 'Snoopy'
                elif BandMax == 'SALT2':  BandMaxText = 'SALT2'


                plt.xlabel('Redshift', fontsize=fontsizePlot)
                plt.ylabel(r'$\mu - \mu_{\rm \Lambda CDM}$', fontsize=fontsizePlot)            

                #--- Plot Title ---

                if PlotTotalMu == False:
                    plt.title('%s-band Hubble Residual from %s'%(BandName, BandMaxText), 
                              fontsize=fontsizePlot+2)

                elif PlotTotalMu == True:
                    if BandsCombination == 'AllBands':
                        if BandMax == 'Bmax' or BandMax == 'Bmax_GP':
                            plt.title('Any YJHK bands Hubble Residual from %s'%(BandMaxText), 
                                      fontsize=fontsizePlot+2)
                        elif BandMax == 'NIRmax':
                            plt.title(r'Any YJHK bands Hubble Residual from $T_{\rm NIR}$ max', 
                                      fontsize=fontsizePlot+2)
                    else: 
                        if BandMax == 'Bmax' or BandMax == 'Bmax_GP':
                            plt.title('%s bands Hubble Residual from %s'%(
                                BandsCombination, BandMaxText), 
                                      fontsize=fontsizePlot+2)
                        elif BandMax == 'NIRmax':
                            plt.title(r'%s bands Hubble Residual from $T_{\rm NIR}$ max'%(
                                BandsCombination), fontsize=fontsizePlot+2)

                #--- Axes ---
                # plt.tick_params(axis='x', labelsize=fontsizePlot)
                # plt.tick_params(axis='y', labelsize=fontsizePlot+2)

                plt.grid(True)
                # plt.tight_layout()

                #----- Save the figures --------

                # In the file name, append 'text' if the labeling the Hubble diagram outliers:
                if labelOutlierLim < 0.7: textLabel = 'label_'
                else: textLabel = '' 

                if   PlotTotalMu == True:  NIRbandsText = BandsCombination+'_'
                elif PlotTotalMu == False: NIRbandsText = ''

                Append_savetext_1 = '' # reset
                if plotErrorBars: Append_savetext_1 = ''
                else: Append_savetext_1 = 'NoBars_'

                if ColorSamples[k] and KindOfData4HD == 'AllSamples':
                    plt.savefig(DirSaveOutput+'Plot_Residual_%s_%s_%s_%s_%s_%s_Color_%s%s%s.png'%(KindOfData4HD, KindOfTemp,
                                KindOfTempSubgroup, chi2_dof_Max_Label, residualMax_Label, zCMB_Min_Label[j], 
                                NIRbandsText, textLabel,Append_savetext_1), dpi=ResolutionPlot_HD)
                else:
                    plt.savefig(DirSaveOutput+'Plot_Residual_%s_%s_%s_%s_%s_%s_%s%s%s.png'%(KindOfData4HD, KindOfTemp,
                                KindOfTempSubgroup, chi2_dof_Max_Label, residualMax_Label, zCMB_Min_Label[j], 
                                NIRbandsText, textLabel,Append_savetext_1), dpi=ResolutionPlot_HD)

                plt.close()
        
textfile7.close()
textfile_8.close()

#----------------------

print "\n # All done."

RMS (z > 0, color=True): 0.13682 (0.10807 WRMS)
 
RMS CSP (z > 0, color=True): 0.13682 (0.10807 WRMS)
 
RMS (z > 0, color=True): 0.13682 (0.10807 WRMS)
 
RMS CSP (z > 0, color=True): 0.13682 (0.10807 WRMS)
 

 # All done.


In [73]:
plt.close();plt.close();plt.close();

In [74]:
print "# All done."

# All done.
